In [22]:
# Fetch thresholds from Snowflake (historical) and today's counts from DWH
try:
    thresholds_df = ret_metabase("egypt", 64923) 


    today_df = dwh_query("Egypt", """
select TO_CHAR(created_at, 'Dy') as day_of_week, 
   TO_CHAR(DATE_TRUNC('hour', created_at) + ((CEIL(EXTRACT(MINUTE FROM created_at) / 30.0) * 30) || ' minutes')::interval, 'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where cast(so.created_at as date) = current_date
group by 1,2
""", "read", columns=[], conn=None)
except Exception as fetch_err:
    print(f"Failed to fetch data sources: {fetch_err}")
    

# Compute current half-hour label and day name
try:
    try:
        tz = pytz.timezone('Africa/Cairo')
    except Exception:
        tz = None
    now = datetime.now(tz) if tz else datetime.now()
    minute = now.minute
    current_hour = now.hour
    current_day_long = now.strftime('%A')
    if minute < 30:
        target_label = f"{current_hour:02d}:00"
    else:
        target_label = f"{current_hour:02d}:30"
    print(f"Target window: {current_day_long} {target_label}")
except Exception as time_err:
    print(f"Failed to compute time window: {time_err}")
    

# Extract today's count from DWH for the target window
try:
    if not isinstance(today_df, pd.DataFrame) or today_df.empty:
        print("No DWH data for today; skipping.")
        
    target_today = today_df[today_df['half_hour_label'] == target_label]
    if target_today.empty:
        print(f"No DWH row for {target_label}; skipping.")
        
    current_order_count = target_today['order_count'].iloc[0]
except Exception as dwh_err:
    print(f"Error parsing DWH data: {dwh_err}")
    

# Extract thresholds from Snowflake for same day and label
try:
    if not isinstance(thresholds_df, pd.DataFrame) or thresholds_df.empty:
        print("Thresholds from Snowflake are empty; skipping.")
    current_day_short = current_day_long[:3]    
    thr_row = thresholds_df[
        (thresholds_df['DAY_OF_WEEK'] == current_day_short) &
        (thresholds_df['HALF_HOUR_LABEL'] == target_label)
    ]
    if thr_row.empty:
        print(f"No threshold row for {current_day_long} {target_label}; skipping.")
        
    lowest_decile = thr_row['LOWEST_DECILE'].iloc[0]
    median_val = thr_row['MEDIAN'].iloc[0]
except Exception as thr_err:
    print(f"Error parsing thresholds: {thr_err}")
    

# Compare and alert
try:
    if pd.notna(current_order_count) and pd.notna(lowest_decile) and current_order_count <= lowest_decile:
        alert_msg = (
            f"<!channel> ⚠️ E-commerce orders have dropped: {current_order_count}. \n "
            f"Average for this time: {median_val}. \n "
            f"Threshold (lowest 10%): {lowest_decile}. \n"
            f"hour: {target_label}."
        )
        
    else:
        print(f"No alert: {current_order_count} vs {lowest_decile} at {current_day_long} {target_label}")
except Exception as alert_err:
    print(f"Error during alert: {alert_err}")



/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
Target window: Sunday 11:00
No alert: 101 vs 57 at Sunday 11:00


In [16]:
thresholds_df.head(20)

,DAY_OF_WEEK,HALF_HOUR_LABEL,MIN_COUNT,MAX_COUNT,MEDIAN,LOWEST_DECILE,LOWEST_FIFTH
0,Sun,21:30,26,533,319.5,191,149
1,Sat,21:00,29,391,267.0,188,172
2,Mon,00:30,27,227,146.5,104,98
3,Thu,08:30,10,71,35.0,24,21
4,Wed,23:30,22,353,168.0,109,100
5,Thu,12:30,12,239,134.0,84,67
6,Thu,14:30,25,321,214.0,120,82
7,Wed,20:00,31,442,265.0,185,157
8,Mon,09:30,11,114,56.0,35,28
9,Wed,13:00,13,340,192.0,133,128


In [20]:
current_day_long

'Sunday'

In [21]:
sund = thresholds_df[
    (thresholds_df['DAY_OF_WEEK'] == current_day_short)]
    
sund

,DAY_OF_WEEK,HALF_HOUR_LABEL,MIN_COUNT,MAX_COUNT,MEDIAN,LOWEST_DECILE,LOWEST_FIFTH
0,Sun,21:30,26,533,319.5,191,149
12,Sun,22:30,31,355,198.5,131,120
14,Sun,08:00,9,57,33.0,23,21
15,Sun,12:00,12,220,123.5,88,78
29,Sun,14:00,19,375,214.5,149,136
31,Sun,10:00,12,107,64.0,46,39
32,Sun,16:00,17,383,249.0,172,160
35,Sun,11:00,20,167,86.0,57,51
44,Sun,21:00,28,411,275.0,178,155
47,Sun,17:30,33,378,243.5,163,141


In [1]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, initialize_env
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import pytz

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
data_to_check = snowflake_query("Egypt", """
select  day_of_week, 
		half_hour_label, 
		min(order_count) min_count, 
		max(order_count) max_count, 
		median(order_count) as median, 
		min(case when percentile >= 0.1 then order_count end) as lowest_decile, 
		min(case when percentile >= 0.05 then order_count end) as lowest_fifth
from (
select *, 
		percent_rank() over(partition by day_of_week, half_hour_label order by order_count) percentile
from (
select *, 
		row_number() over(partition by day_of_week, half_hour_label order by order_count) r 
from (
select dayname(so.created_at) as day_of_week, 
	   so.created_at::date as so_date, 
	   TO_CHAR( DATEADD(minute,CEIL(DATE_PART(minute, created_at) / 30.0) * 30,DATE_TRUNC(hour, created_at)),'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where so.created_at::date between current_date - 900 and current_date 
group by all)))
group by all """)


data = dwh_query("Egypt", """
select TO_CHAR(created_at, 'Dy') as day_of_week, 
	   TO_CHAR(DATE_TRUNC('hour', created_at) + ((CEIL(EXTRACT(MINUTE FROM created_at) / 30.0) * 30) || ' minutes')::interval, 'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where cast(so.created_at as date) = current_date
group by 1,2
""", "read", columns=[], conn=None)


target_check = data[data['half_hour_label'] == target_label]
confirm_data = data_to_check[(data_to_check['day_of_week'] == target_check['day_of_week'].iloc[0])
                            & (data_to_check['half_hour_label'] == target_label)]
if targert_check['order_count'].iloc[0] <= confirm_data['lowest_decile'].iloc[0]:
    #send to slack the alert 

/home/ec2-user/service_account_key.json


,day_of_week,half_hour_label,min_count,max_count,median,lowest_decile,lowest_fifth
0,Sat,11:30,29,177,103.500,75,69
1,Fri,21:30,32,235,150.000,107,95
2,Tue,07:00,11,59,33.500,22,17
3,Sat,23:30,30,288,173.500,111,97
4,Thu,08:00,11,66,30.000,21,19
...,...,...,...,...,...,...,...
331,Sun,04:30,23,91,50.500,35,32
332,Wed,22:00,32,782,232.500,140,124
333,Thu,18:30,28,305,221.000,87,59
334,Wed,12:00,7,359,128.500,96,84


In [14]:
data = dwh_query("Egypt", """
select TO_CHAR(created_at, 'Dy') as day_of_week, 
	   TO_CHAR(DATE_TRUNC('hour', created_at) + ((CEIL(EXTRACT(MINUTE FROM created_at) / 30.0) * 30) || ' minutes')::interval, 'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where cast(so.created_at as date) = current_date
group by 1,2
""", "read", columns=[], conn=None)


target_check = data[data['half_hour_label'] == target_label]
confirm_data = data_to_check[(data_to_check['day_of_week'] == target_check['day_of_week'].iloc[0])
                            & (data_to_check['half_hour_label'] == target_label)]
if targert_check['order_count'].iloc[0] <= confirm_data['lowest_decile'].iloc[0]:
    #send to slack the alert 

/home/ec2-user/service_account_key.json


,day_of_week,half_hour_label,order_count
0,Tue,00:00,6
1,Tue,00:30,111
2,Tue,01:00,117
3,Tue,01:30,95
4,Tue,02:00,86
5,Tue,02:30,70
6,Tue,03:00,66
7,Tue,03:30,61
8,Tue,04:00,50
9,Tue,04:30,39


In [13]:
# Use Cairo timezone for alignment with business hours
try:
    tz = pytz.timezone('Africa/Cairo')
except Exception:
    tz = None
now = datetime.now(tz) if tz else datetime.now()

current_hour = now.hour
minute = now.minute
print(f"Current time: {now} | Hour: {current_hour} | Minute: {minute}")

# Always select the previous half-hour window
if minute < 30:
    target_label = f"{current_hour:02d}:00"
else:
    target_label = f"{current_hour:02d}:30"

print(f"Computed target half-hour label: {target_label}")



Current time: 2025-10-07 10:14:28.454230+03:00 | Hour: 10 | Minute: 14
Computed target half-hour label: 10:00


In [16]:
target_check = data[data['half_hour_label'] == target_label]
confirm_data = data_to_check[(data_to_check['day_of_week'] == target_check['day_of_week'].iloc[0])
                            & (data_to_check['half_hour_label'] == target_label)]
if targert_check['order_count'].iloc[0] <= confirm_data['lowest_decile'].iloc[0]:
    #send to slack the alert 

,day_of_week,half_hour_label,order_count
20,Tue,10:00,62


In [21]:
confirm_data = data_to_check[(data_to_check['day_of_week'] == target_check['day_of_week'].iloc[0])
                            & (data_to_check['half_hour_label'] == target_label)]
if targert_check['order_count'].iloc[0] <= confirm_data['lowest_decile'].iloc[0]:
    #send to slack the alert 

,day_of_week,half_hour_label,min_count,max_count,median,lowest_decile,lowest_fifth
227,Tue,10:00,8,165,69.000,44,40


In [11]:
    last_half_hour = dwh_query("Egypt", """
with thresholds as (
select	day_of_week,
		half_hour_label,
		min(order_count) as min_count,
		max(order_count) as max_count,
		percentile_disc(0.5) within group (order by order_count) as median,
		percentile_disc(0.1) within group (order by order_count) as lowest_decile,
		percentile_disc(0.05) within group (order by order_count) as lowest_fifth
from (
	select	to_char(so.created_at, 'FMDay') as day_of_week,
			extract(hour from so.created_at)::int as hour,
			so.created_at::date as so_date,
			to_char(
				date_trunc('hour', so.created_at)
				+
				interval '1 minute' * (ceil(extract(minute from so.created_at) / 30.0)::int * 30),
				'HH24:MI'
			) as half_hour_label,
			count(distinct parent_sales_order_id) as order_count
	from public.sales_orders so
	where so.created_at::date between current_date - 60 and current_date
	group by to_char(so.created_at, 'FMDay'), extract(hour from so.created_at), so.created_at::date, to_char(
			date_trunc('hour', so.created_at)
			+
			interval '1 minute' * (ceil(extract(minute from so.created_at) / 30.0)::int * 30),
			'HH24:MI'
		)
)s
group by day_of_week, half_hour_label
),

so as (
select	to_char(so.created_at, 'FMDay') as day_of_week,
		extract(hour from so.created_at)::int as hour,
		so.created_at::date as so_date,
		to_char(
			date_trunc('hour', so.created_at)
			+
			interval '1 minute' * (ceil(extract(minute from so.created_at) / 30.0)::int * 30),
			'HH24:MI'
		) as half_hour_label,
		count(distinct parent_sales_order_id) as order_count
from public.sales_orders so
where so.created_at::date = current_date
group by to_char(so.created_at, 'FMDay'), extract(hour from so.created_at), so.created_at::date, to_char(
		date_trunc('hour', so.created_at)
		+
		interval '1 minute' * (ceil(extract(minute from so.created_at) / 30.0)::int * 30),
		'HH24:MI'
)
) 

select t.*,
	   so.order_count
from thresholds t
left join so on t.day_of_week = so.day_of_week and t.half_hour_label = so.half_hour_label
where t.day_of_week = to_char(current_date, 'FMDay')
order by t.half_hour_label

""", "read", columns=[], conn=None)
last_half_hour

/home/ec2-user/service_account_key.json


,day_of_week,half_hour_label,min_count,max_count,median,lowest_decile,lowest_fifth,order_count
0,Monday,00:00,1,149,9,3,1,3.0
1,Monday,00:30,99,135,109,99,99,101.0
2,Monday,01:00,1,143,7,2,1,91.0
3,Monday,01:00,1,143,7,2,1,6.0
4,Monday,01:30,76,160,91,76,76,83.0
5,Monday,02:00,2,140,4,2,2,89.0
6,Monday,02:00,2,140,4,2,2,3.0
7,Monday,02:30,59,92,73,59,59,64.0
8,Monday,03:00,1,93,58,2,1,72.0
9,Monday,03:30,50,79,63,50,50,57.0


In [10]:
df_filtered = last_half_hour[last_half_hour['half_hour_label'] == "15:00"]
df_filtered

,day_of_week,half_hour_label,min_count,max_count,median,lowest_decile,lowest_fifth,order_count
41,Monday,15:00,1,256,9,2,1,152.0
42,Monday,15:00,1,256,9,2,1,1.0


In [7]:
df_filtered = last_half_hour[last_half_hour['half_hour_label'] == "15:30"]
df_filtered

,day_of_week,hour,half_hour_label,min_count,max_count,median,lowest_decile,lowest_fifth,order_count
70,Monday,15.0,15:30,112,210,185.0,160,160,112.0


In [18]:
try:
    if isinstance(last_half_hour, pd.DataFrame) and not last_half_hour.empty:
        # Use Cairo timezone for alignment with business hours
        try:
            tz = pytz.timezone('Africa/Cairo')
        except Exception:
            tz = None
        now = datetime.now(tz) if tz else datetime.now()

        current_hour = now.hour
        minute = now.minute
        print(f"Current time: {now} | Hour: {current_hour} | Minute: {minute}")

        if minute < 30:
            target_label = f"{current_hour:02d}:00"
        else:
            target_label = f"{current_hour:02d}:30"

        print(f"Computed target half-hour label: {target_label}")

        # Filter for matching half-hour label and current hour to avoid duplicates
        try:
            df_filtered = last_half_hour[
                (last_half_hour['half_hour_label'] == target_label) &
                (last_half_hour['hour'].astype(int) == int(current_hour))
            ]
        except Exception:
            # Fallback if type casting fails
            df_filtered = last_half_hour[last_half_hour['half_hour_label'] == target_label]

        print(f"Matched rows for label '{target_label}' and hour '{current_hour}': {len(df_filtered)}")

        if not df_filtered.empty:
            row = df_filtered.iloc[0]
            order_count = row.get("order_count")
            lowest_decile = row.get("lowest_decile")
            print(f"Selected row -> order_count: {order_count}, lowest_decile: {lowest_decile}")

            if pd.notna(order_count) and pd.notna(lowest_decile) and order_count <= lowest_decile:
                print(
                    f"⚠️ E-commerce orders have dropped: {order_count} <= lowest decile {lowest_decile} "
                    f"for half-hour {target_label} (hour {current_hour})."
                )
                try:
                    send_text_slack(channel='seif_error_logs', text=alert_msg)
                except Exception as slack_err:
                    print(f"Failed to send Slack alert: {slack_err}")
            else:
                print(
                    f"No alert: order_count {order_count} vs lowest_decile {lowest_decile} for {target_label} hour {current_hour}"
                )
        else:
            print(f"No matching row for label {target_label} and hour {current_hour}; skipping alert.")
    else:
        print("last_half_hour is empty or not a DataFrame; skipping alert check.")
except Exception as e:
    print(f"Error while evaluating last_half_hour alert condition: {e}")


Current time: 2025-10-05 12:42:18.079330+03:00 | Hour: 12 | Minute: 42
Computed target half-hour label: 12:30
Matched rows for label '12:30' and hour '12': 1
Selected row -> order_count: 35.0, lowest_decile: 99
⚠️ E-commerce orders have dropped: 35.0 <= lowest decile 99 for half-hour 12:30 (hour 12).
Failed to send Slack alert: name 'send_text_slack' is not defined


In [7]:

from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query
import pandas as pd

from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import pytz





def send_text_slack(channel, text):
    """
    Send a text message to a Slack channel.
    
    Args:
        channel (str): The Slack channel name to send the message to
        text (str): The message text to send
        
    Raises:
        Exception: If the message fails to send
    """
    print(f"Sending Slack message to channel '{channel}'...")
    
    import os
    try:
        import slack
    except Exception as import_err:
        print(f"Slack SDK not available: {import_err}")
        return

    slack_token = os.environ.get("SLACK_TOKEN")
    if not slack_token:
        print("SLACK_TOKEN not set; skipping Slack notification.")
        return

    client = slack.WebClient(token=slack_token)
    try:
        client.chat_postMessage(channel=channel, text=text)
        print('Slack message sent successfully')
    except Exception as e:
        print(f'Failed to send Slack message: {str(e)}')


def task_fail_slack_alert(context):
    """
    Send a Slack alert when an Airflow task fails.
    
    This function is called by Airflow's on_failure_callback and formats
    the error information into a readable Slack message.
    
    Args:
        context (dict): Airflow context containing task failure information
    """
    print("Preparing Slack failure alert...")
    
    slack_msg = """
        :red_circle: Task Failed.
        *Task*: {task}  
        *Dag*: {dag} 
        *Execution Time*: {exec_date}  
        *Reason*: {exception}
    """.format(
        task=context.get('task_instance').task_id,
        dag=context.get('task_instance').dag_id,
        exec_date=context.get('execution_date'),
        exception=context.get('exception')
    )

    send_text_slack(channel='ecom_alert', text=slack_msg)



# Fetch thresholds from Snowflake (historical) and today's counts from DWH
try:
    thresholds_df = snowflake_query("Egypt", """
select  day_of_week, 
    half_hour_label, 
    min(order_count) min_count, 
    max(order_count) max_count, 
    median(order_count) as median, 
    min(case when percentile >= 0.1 then order_count end) as lowest_decile, 
    min(case when percentile >= 0.05 then order_count end) as lowest_fifth
from (
select *, 
    percent_rank() over(partition by day_of_week, half_hour_label order by order_count) percentile
from (
select *, 
    row_number() over(partition by day_of_week, half_hour_label order by order_count) r 
from (
select dayname(so.created_at) as day_of_week, 
   so.created_at::date as so_date, 
   TO_CHAR( DATEADD(minute,CEIL(DATE_PART(minute, created_at) / 30.0) * 30,DATE_TRUNC(hour, created_at)),'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where so.created_at::date between current_date - 900 and current_date 
group by all)))
group by all
order by 1, 2
""")

    today_df = dwh_query("Egypt", """
select TO_CHAR(created_at, 'Dy') as day_of_week, 
   TO_CHAR(DATE_TRUNC('hour', created_at) + ((CEIL(EXTRACT(MINUTE FROM created_at) / 30.0) * 30) || ' minutes')::interval, 'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where cast(so.created_at as date) = current_date
group by 1,2
""", "read", columns=[], conn=None)
except Exception as fetch_err:
    print(f"Failed to fetch data sources: {fetch_err}")
    

# Compute current half-hour label and day name
try:
    try:
        tz = pytz.timezone('Africa/Cairo')
    except Exception:
        tz = None
    now = datetime.now(tz) if tz else datetime.now()
    minute = now.minute
    current_hour = now.hour
    current_day_long = now.strftime('%A')
    if minute < 30:
        target_label = f"{current_hour:02d}:00"
    else:
        target_label = f"{current_hour:02d}:30"
    print(f"Target window: {current_day_long} {target_label}")
except Exception as time_err:
    print(f"Failed to compute time window: {time_err}")
    

#     # Extract today's count from DWH for the target window
#     try:
#         if not isinstance(today_df, pd.DataFrame) or today_df.empty:
#             print("No DWH data for today; skipping.")
#             return None
#         target_today = today_df[today_df['half_hour_label'] == target_label]
#         if target_today.empty:
#             print(f"No DWH row for {target_label}; skipping.")
#             return None
#         current_order_count = target_today['order_count'].iloc[0]
#     except Exception as dwh_err:
#         print(f"Error parsing DWH data: {dwh_err}")
#         return None

#     # Extract thresholds from Snowflake for same day and label
#     try:
#         if not isinstance(thresholds_df, pd.DataFrame) or thresholds_df.empty:
#             print("Thresholds from Snowflake are empty; skipping.")
#             return None
#         thr_row = thresholds_df[
#             (thresholds_df['day_of_week'] == current_day_long) &
#             (thresholds_df['half_hour_label'] == target_label)
#         ]
#         if thr_row.empty:
#             print(f"No threshold row for {current_day_long} {target_label}; skipping.")
#             return None
#         lowest_decile = thr_row['lowest_decile'].iloc[0]
#         median_val = thr_row['median'].iloc[0]
#     except Exception as thr_err:
#         print(f"Error parsing thresholds: {thr_err}")
#         return None

#     # Compare and alert
#     try:
#         if pd.notna(current_order_count) and pd.notna(lowest_decile) and current_order_count <= lowest_decile:
#             alert_msg = (
#                 f"<!channel> ⚠️ E-commerce orders have dropped: {current_order_count}. \n "
#                 f"Average for this time: {median_val}. \n "
#                 f"Threshold (lowest 10%): {lowest_decile}. \n"
#                 f"hour: {target_label}."
#             )
#             send_text_slack(channel='ecom_alert', text=alert_msg)
#         else:
#             print(f"No alert: {current_order_count} vs {lowest_decile} at {current_day_long} {target_label}")
#     except Exception as alert_err:
#         print(f"Error during alert: {alert_err}")

#     return None


/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json
Target window: Monday 11:00


In [17]:
thresholds_df = ret_metabase("egypt", 64923) 
thresholds_df

/home/ec2-user/service_account_key.json


,DAY_OF_WEEK,HALF_HOUR_LABEL,MIN_COUNT,MAX_COUNT,MEDIAN,LOWEST_DECILE,LOWEST_FIFTH
0,Fri,21:30,32,235,150.0,107,95
1,Thu,08:00,11,66,30.0,21,18
2,Tue,00:00,33,3053,187.0,122,113
3,Thu,04:30,21,82,46.0,30,26
4,Wed,14:30,23,522,241.0,174,149
...,...,...,...,...,...,...,...
331,Mon,02:30,22,160,90.0,63,59
332,Fri,01:30,16,196,87.0,63,57
333,Tue,06:00,16,73,36.0,23,21
334,Wed,14:00,18,515,222.0,153,139


In [14]:
thresholds_df.to_excel("data.xlsx", index=False)

In [32]:
data = """
112321, 117589, 125739, 154487, 155553, 158267, 165019, 166634, 20270, 213696, 213940, 217685, 225032, 229136, 229627, 22981, 236988, 238924, 250955, 252990, 253867, 269205, 269960, 270860, 304879, 314291, 31793, 318758, 322558, 329248, 360183, 362903, 374103, 374798, 380073, 383883, 38788, 389398, 398865, 402952, 405872, 426257, 455405, 476073, 484728, 491299, 526732, 556957, 564543, 572768, 591903, 609420, 638675, 642851, 649257, 669531, 675801, 70083, 703009, 708214, 741303, 755700, 757972, 762446, 765332, 776484, 785968, 79543, 79859, 846614, 849214, 857250, 862020, 873097, 875596, 878207, 881669, 885433, 889385, 904006, 921271, 923065, 93633, 940952, 99751
111766, 11313, 124805, 125924, 135642, 151157, 157723, 164500, 176435, 184165, 189730, 19206, 193573, 194304, 197000, 199980, 210999, 213396, 234081, 243279, 244870, 255403, 26188, 270389, 272353, 275225, 28883, 293452, 308818, 315716, 321357, 330543, 358901, 364722, 365046, 366123, 372821, 383521, 411657, 422063, 43001, 437798, 444840, 45688, 469130, 476126, 480679, 496712, 539962, 541309, 541774, 5671, 570073, 592353, 596316, 618030, 654339, 657310, 663521, 664765, 695005, 707933, 716981, 730126, 776618, 779630, 853317, 858160, 864365, 867663, 916607, 919114
162291, 165322, 17826, 180221, 190313, 193453, 201024, 230885, 233786, 239600, 245143, 256773, 269778, 291822, 316201, 358411, 359838, 365827, 374956, 380421, 410940, 436175, 455816, 46109, 485309, 492521, 504329, 532069, 537344, 546588, 551248, 596777, 604745, 607403, 632791, 683361, 685973, 690104, 716655, 740583, 755911, 761483, 766975, 772241, 868635, 868703, 879653, 911075, 913591, 925699, 933330, 937945
102738, 10796, 118534, 120828, 123662, 216112, 21646, 219910, 225413, 226955, 233793, 24820, 359077, 412247, 42756, 429833, 543112, 5452, 578420, 618109, 644160, 65154, 719409, 733741, 84452, 861974, 877107, 931198, 934593
133245, 176878, 182877, 194915, 250019, 276264, 300410, 305131, 339181, 343236, 369291, 394892, 400039, 402193, 403409, 464192, 574195, 57426, 57455, 579228, 585627, 590169, 627245, 672283, 682196, 688373, 707040, 767673, 926295
134215, 18344, 19015, 211567, 25869, 362562, 374874, 454248, 491115, 499522, 556187, 562742, 563738, 594682, 632711, 700774, 763528
103485, 110663, 151555, 177085, 18696, 224415, 237147, 321758, 371636, 449887, 490553, 513422, 603042, 705216, 72881, 776574, 875595
381895
10611, 154074, 161652, 202633, 224468, 232643, 328429, 344707, 366810, 383836, 43224, 601691, 69283, 785045
182503, 200272, 335383, 357719, 430099, 448455, 453481, 563592, 579249, 662950, 670537
11335, 127441, 133653, 285882, 357722, 360599, 36945, 392295, 397095, 474885, 543581, 707175, 873618, 913351
310233, 360239, 391872, 541747, 669075
121059, 180735, 187568, 290407, 291680, 665254, 768385, 88402
181430, 224909, 324935, 400649, 454907
210147, 380750, 649581
235149, 246131, 518808, 526712, 563869, 636178
10316, 581367, 783712
164356
637962, 782560, 881999
218332, 218622, 349870, 669954
240546, 323981, 337756, 740147
100307, 101776, 102761, 106542, 108600, 109399, 112489, 114667, 114779, 115585, 117603, 118149, 119900, 124793, 125537, 128592, 129469, 130118, 130634, 14084, 149760, 149777, 150774, 151631, 154032, 155098, 155099, 155492, 15605, 157788, 159433, 161621, 163308, 164633, 165820, 166370, 166437, 167335, 17051, 176161, 176590, 18217, 182540, 183064, 185171, 188165, 189344, 189379, 192907, 193709, 195615, 200633, 202910, 20654, 208087, 209249, 211444, 213712, 214346, 216279, 221330, 222003, 223275, 223530, 226432, 232427, 234667, 235389, 237630, 238258, 238670, 238885, 242538, 24355, 244136, 245434, 245899, 246535, 249252, 250243, 252203, 254283, 25489, 262347, 266093, 266986, 268300, 268934, 269493, 269690, 270059, 270874, 272417, 272532, 273091, 273552, 273794, 274981, 277118, 278848, 279342, 280793, 282640, 28400, 286992, 287560, 2877, 290072, 292878, 297982, 300291, 302326, 306751, 310671, 311529, 312479, 313112, 314374, 314610, 317896, 31997, 322857, 325214, 328522, 328526, 331614, 331676, 332867, 335616, 336162, 336573, 337522, 338088, 339013, 341393, 343451, 343835, 344670, 346302, 346894, 346906, 348408, 352538, 353940, 354434, 355922, 357181, 357232, 360648, 362091, 364059, 365138, 365262, 366650, 367369, 368588, 369035, 371235, 371757, 371976, 372879, 373394, 373870, 37859, 380107, 380451, 382745, 385173, 38737, 390905, 391090, 391776, 391901, 395869, 396447, 396500, 396803, 39858, 399410, 400630, 402248, 402990, 405501, 405918, 418078, 426574, 429990, 441315, 441332, 442486, 443957, 451084, 454206, 454434, 454791, 455379, 457336, 465367, 466221, 469269, 47328, 474150, 474259, 474384, 474878, 475490, 4806, 482253, 487130, 489136, 491038, 493349, 493793, 496998, 497441, 499388, 502123, 502445, 504891, 506763, 512826, 515180, 524494, 525678, 527260, 52813, 530631, 532093, 535890, 535983, 544260, 546129, 548070, 548329, 549991, 552390, 554294, 555116, 556275, 557435, 557517, 557910, 558642, 560221, 561864, 564081, 565136, 569163, 572064, 576220, 578400, 579276, 579753, 580326, 580356, 583605, 583777, 594575, 596635, 605810, 617147, 621499, 623753, 62429, 624353, 628607, 631486, 632079, 634021, 634424, 634678, 636238, 641244, 642983, 644758, 64480, 648483, 648879, 64938, 650956, 65182, 655490, 668561, 670562, 670983, 671179, 67267, 682773, 682781, 682924, 686923, 691041, 693035, 693540, 695797, 70032, 708073, 714112, 715678, 718625, 723548, 739380, 742510, 744048, 745182, 748469, 750344, 750857, 75109, 756867, 758494, 76004, 76035, 760917, 763155, 763418, 767347, 770338, 773501, 774704, 775498, 777110, 780579, 781138, 783137, 786262, 80409, 85255, 853755, 854072, 857288, 858028, 862366, 864559, 866751, 869843, 874773, 877768, 878706, 889601, 890710, 893291, 89567, 904828, 905466, 909567, 909813, 912894, 914072, 918303, 919150, 921551, 922209, 924995, 929859, 930391, 930698, 930893, 932092, 932276, 932871, 934816, 935478, 936595, 939051, 939709, 939906, 940003, 943436, 98899
307879, 439835, 669808, 855596, 923077
133024, 490911
275092, 293359, 534881, 578011
212269
607379
398034, 543538
100958, 101146, 102118, 102265, 103649, 104920, 104986, 10528, 105804, 107455, 107895, 108132, 108486, 108791, 109097, 109375, 109697, 110306, 112229, 112492, 112887, 115156, 116312, 11646, 117041, 117552, 118712, 121713, 122205, 12476, 126516, 126722, 128776, 129852, 130328, 130794, 132775, 133333, 133575, 13522, 13746, 138112, 141558, 14401, 144624, 149709, 150333, 151607, 151728, 153183, 153686, 154743, 154957, 156301, 156619, 156844, 157253, 157732, 159106, 159395, 160938, 161878, 162017, 162317, 162706, 162730, 163386, 16368, 163917, 164316, 165525, 166329, 175437, 175827, 176187, 176454, 177253, 178384, 180078, 181659, 181727, 181818, 181920, 18243, 182999, 183038, 183426, 184500, 185572, 186340, 18668, 186973, 187443, 18748, 188084, 188980, 189664, 189763, 190203, 192501, 194369, 194746, 194754, 194918, 195165, 195625, 197846, 199293, 199839, 199911, 200231, 200517, 200825, 201172, 201256, 201457, 202104, 205518, 206469, 20754, 207919, 209395, 211360, 211616, 214222, 214359, 215001, 215803, 217503, 220737, 221328, 22201, 22241, 223375, 223407, 223962, 22454, 225216, 225374, 226100, 226977, 227503, 229303, 229838, 229860, 230430, 231367, 231403, 231946, 233488, 233760, 234212, 234227, 234289, 234297, 234357, 234880, 234912, 235036, 235352, 235505, 235852, 235893, 237127, 237548, 237639, 237868, 238578, 24108, 241504, 243782, 24409, 244220, 244839, 245262, 246876, 247363, 248070, 248521, 249354, 250362, 25101, 251518, 251788, 251822, 252389, 253106, 255833, 256107, 257022, 258080, 25844, 26065, 260815, 260944, 262537, 263605, 263740, 266382, 266550, 267217, 267947, 268186, 268531, 269718, 27044, 272381, 272622, 27611, 276216, 278266, 282055, 28521, 287985, 288656, 289661, 290112, 290278, 290359, 29098, 293150, 29531, 295854, 296019, 296222, 296609, 302259, 303409, 304200, 304321, 304894, 304957, 305058, 308280, 309643, 311304, 314638, 314796, 315034, 315087, 31695, 317692, 318448, 321688, 32327, 324237, 326245, 32728, 327571, 327788, 328030, 328299, 329420, 329740, 330989, 331480, 333738, 334125, 334508, 334957, 336501, 337457, 340656, 341295, 341404, 341669, 341832, 343242, 343570, 344245, 344599, 345899, 347049, 347170, 347925, 348417, 349264, 34958, 350383, 353104, 353698, 355355, 355942, 356558, 356735, 357211, 358121, 358962, 359298, 359458, 359680, 359692, 35985, 359966, 359968, 360505, 361994, 362018, 362163, 362856, 362956, 36339, 363838, 364174, 36457, 365512, 367440, 369169, 369956, 370037, 370872, 371427, 37368, 374191, 375458, 376526, 376830, 377049, 377471, 380184, 380450, 380479, 380483, 380509, 380639, 380654, 381863, 382414, 383129, 383489, 384155, 384450, 384713, 386084, 386159, 38620, 386228, 387152, 388098, 389320, 38936, 389941, 390008, 39098, 39144, 392259, 392922, 394051, 394296, 395620, 395665, 395782, 397289, 397732, 398022, 398199, 398505, 398684, 399656, 400268, 400319, 400625, 401263, 40150, 401565, 403098, 403877, 404191, 404378, 404938, 405414, 406937, 408230, 41012, 413020, 413778, 414976, 41526, 418804, 42046, 420502, 420765, 42400, 424536, 426885, 429347, 433015, 433756, 434143, 434200, 436013, 436028, 437079, 438589, 440906, 443289, 443770, 445510, 445969, 451653, 451999, 453762, 454195, 454216, 45496, 457694, 465133, 465164, 46604, 466655, 467387, 46868, 468905, 46928, 469559, 473073, 474169, 47510, 476179, 477334, 479511, 481260, 48246, 482586, 485867, 486326, 486359, 487119, 488961, 490070, 490210, 491306, 491413, 491747, 492394, 494644, 496694, 496909, 49707, 497726, 498061, 4991, 503767, 503856, 503956, 503968, 504026, 504397, 505016, 505259, 506750, 508265, 509653, 512524, 51352, 51404, 5156, 516118, 51623, 51756, 518835, 519736, 52308, 523260, 524442, 525733, 526244, 531578, 532352, 537797, 539430, 53996, 541540, 542388, 544458, 545600, 545633, 546325, 548673, 549333, 551678, 552401, 553216, 554623, 556557, 556579, 557253, 558731, 559970, 560650, 560685, 561361, 561428, 561765, 561850, 561859, 562169, 562414, 562523, 562729, 562804, 564095, 564161, 564609, 564724, 566440, 566469, 566675, 567735, 569259, 570375, 571338, 575806, 575868, 576715, 577163, 577414, 578179, 579827, 581217, 581858, 58305, 58443, 584565, 588267, 592582, 593010, 594634, 594654, 595391, 59606, 596720, 597256, 597362, 598223, 598663, 599506, 599865, 600691, 601265, 602132, 602821, 60656, 606952, 608429, 608742, 609631, 610331, 610765, 618041, 61948, 620182, 620290, 622610, 623806, 628862, 629980, 631621, 631719, 63281, 634468, 635338, 635695, 635922, 636335, 636936, 640300, 640307, 640403, 641657, 643112, 644645, 644658, 645006, 64561, 646559, 648827, 650401, 657243, 657593, 658671, 659344, 659402, 662146, 66252, 66470, 666475, 66826, 669658, 670151, 672903, 673964, 674431, 675923, 67686, 677881, 679855, 680200, 683125, 685592, 686329, 687025, 687496, 688824, 689663, 691270, 69129, 692064, 693995, 694358, 694512, 697148, 69733, 697330, 697536, 697783, 699381, 699738, 70778, 70873, 710264, 711639, 71175, 713060, 713783, 716039, 717764, 71965, 721537, 721837, 722098, 723613, 724097, 726920, 72748, 727827, 731535, 733982, 735116, 735141, 735288, 736305, 736892, 738040, 738331, 739467, 740043, 740991, 746364, 748000, 748809, 749250, 754406, 754505, 755551, 756060, 757960, 75805, 760501, 761025, 761115, 761140, 762067, 762248, 76249, 763938, 764090, 764344, 765797, 766413, 766864, 768536, 769323, 77031, 770670, 770750, 771753, 773808, 774228, 774503, 777344, 777918, 779839, 780443, 78045, 78673, 787063, 787087, 787612, 788038, 79077, 793194, 796582, 79890, 80380, 805090, 81093, 82313, 8240, 82769, 84017, 84384, 846509, 850866, 851727, 852234, 85330, 85415, 855469, 855942, 858516, 859490, 859545, 859620, 860914, 862700, 862875, 863629, 866492, 868920, 871873, 871910, 872811, 873233, 874257, 874945, 875299, 875439, 876548, 878197, 878457, 878841, 87998, 880726, 882672, 884621, 885072, 88547, 88748, 88899, 893095, 893991, 90104, 903996, 907139, 907816, 907957, 913285, 919226, 919582, 920844, 923718, 927860, 92901, 929721, 930968, 931549, 931941, 934441, 934519, 935552, 935744, 936187, 937861, 93959, 940147, 940919, 94322, 9613, 9664, 97024, 9779, 9826, 99509
704488
187659, 289268
576012
315523, 451974, 84177
243524
100220, 102091, 103168, 103557, 104099, 104252, 10454, 104588, 106820, 107055, 107081, 10874, 109613, 111087, 111820, 112282, 112358, 113065, 113613, 115454, 115499, 115964, 116428, 117516, 118438, 118688, 122403, 12295, 124144, 125476, 130336, 132719, 132809, 133317, 133445, 134018, 134409, 13526, 135261, 135364, 13759, 14418, 147895, 150459, 151085, 151091, 151282, 151291, 152344, 152629, 153160, 153539, 153874, 154553, 155014, 155028, 155230, 155810, 156340, 157009, 157054, 157062, 157595, 15763, 158938, 159035, 159040, 160916, 161053, 161419, 162244, 162524, 163769, 164612, 16562, 165751, 167027, 16955, 175147, 17524, 175749, 17622, 176314, 176346, 176607, 176741, 177484, 179606, 179852, 180896, 181368, 181460, 181706, 182871, 18318, 183263, 18327, 185105, 185654, 18877, 188974, 18899, 189797, 190052, 192985, 193224, 193635, 193750, 193894, 194141, 194262, 194291, 194383, 19506, 195083, 19534, 195549, 197924, 198578, 198588, 199482, 199608, 19967, 200346, 200631, 200655, 20183, 201885, 20332, 205050, 205377, 206299, 20643, 207281, 207788, 207964, 208683, 209316, 209769, 212466, 21257, 213771, 215687, 21703, 21813, 218361, 218743, 218892, 22024, 22178, 222280, 223217, 223747, 226144, 226162, 229760, 229993, 230058, 230868, 231276, 231392, 232423, 232861, 23303, 233885, 234214, 23449, 234624, 23487, 235023, 235832, 235935, 236964, 238988, 23914, 239543, 240997, 241522, 242891, 243477, 244290, 246018, 246467, 24650, 247072, 247654, 248257, 249441, 250395, 250579, 252252, 252952, 253312, 25335, 253933, 254766, 255053, 255210, 255652, 256287, 257598, 25896, 259494, 261856, 264279, 264852, 265513, 266730, 266855, 26722, 269239, 269527, 27098, 27469, 275673, 27682, 277139, 277429, 28009, 28133, 281535, 284631, 287860, 288090, 28835, 29035, 290991, 291954, 293975, 295727, 2975, 297923, 299264, 299986, 300247, 300697, 301403, 302278, 302360, 30510, 307298, 308162, 309531, 310000, 312425, 312428, 313073, 313702, 314035, 315309, 315641, 322060, 32340, 324439, 324776, 326594, 327465, 328787, 329157, 331828, 336382, 337551, 337752, 338595, 338936, 340045, 340741, 343056, 344048, 344243, 346352, 347628, 348147, 349005, 349496, 350364, 350372, 350626, 351996, 35300, 353084, 355197, 356696, 357108, 357324, 35777, 358643, 35974, 360073, 360565, 363008, 364202, 366026, 367787, 368421, 368827, 369034, 372154, 372175, 373805, 374282, 374335, 374376, 375024, 375758, 37678, 377418, 377702, 378513, 380755, 387202, 388233, 388302, 38865, 388677, 389938, 390202, 390212, 390735, 39123, 391607, 394459, 39491, 395426, 395832, 395874, 39646, 396739, 396765, 397043, 397354, 399825, 400296, 40041, 400431, 400533, 401431, 40370, 404266, 40467, 404720, 404839, 404870, 405509, 40575, 405773, 406007, 409141, 409712, 409811, 410172, 413747, 414424, 41715, 42031, 420381, 421793, 423863, 424656, 42536, 426976, 427457, 430154, 432628, 434019, 437554, 43782, 439490, 44018, 442647, 44384, 444156, 44549, 449504, 453660, 455696, 456141, 456779, 463444, 466084, 466621, 467091, 467912, 469640, 469953, 470834, 473018, 473194, 474577, 478200, 479526, 482278, 484588, 486991, 487031, 488582, 491301, 494232, 496415, 496788, 496790, 496792, 496846, 497276, 497641, 498016, 499440, 500801, 500854, 50129, 503473, 503595, 503765, 504394, 505930, 506954, 50877, 510276, 51038, 510745, 519076, 519934, 520145, 520763, 521721, 527137, 530699, 530830, 531000, 531895, 534923, 536010, 536087, 538648, 540070, 540191, 54033, 542549, 542961, 54341, 546415, 547328, 54781, 548305, 548472, 548746, 550257, 550778, 551310, 552863, 552962, 553315, 553661, 55368, 55432, 555294, 555853, 557874, 558596, 559870, 560021, 560190, 56214, 562539, 562974, 563124, 563366, 563770, 564976, 565787, 566476, 568304, 57277, 576013, 576523, 57713, 578597, 579099, 580109, 580945, 58181, 582229, 582837, 58299, 584665, 586759, 588063, 588845, 59167, 592219, 592670, 593576, 593732, 59415, 59481, 597434, 597488, 597577, 598141, 599374, 59971, 600092, 601714, 603199, 604978, 606931, 611144, 617967, 61827, 620169, 625506, 632148, 63215, 63528, 635476, 636889, 636951, 637713, 638403, 639226, 640161, 642082, 642975, 643331, 644427, 645351, 645883, 646887, 646992, 648024, 648132, 651677, 651827, 652547, 652870, 653097, 654220, 656107, 661606, 664197, 664958, 66580, 66640, 66668, 666819, 66725, 66766, 66908, 671112, 671502, 67211, 67268, 673400, 673480, 675823, 676025, 678983, 679312, 679542, 680568, 680949, 68105, 681640, 685709, 685946, 6935, 695657, 697036, 697094, 6971, 699525, 70138, 702349, 70284, 702886, 704050, 704635, 705225, 709953, 709970, 711069, 711679, 712494, 712727, 713869, 715088, 715925, 71694, 717620, 717920, 719823, 720983, 723410, 723902, 72469, 726800, 729043, 729983, 734304, 73443, 734601, 735409, 735933, 736015, 73750, 737621, 738621, 739132, 73947, 740366, 742059, 74269, 745568, 748212, 74933, 751510, 752654, 753058, 753097, 753855, 754313, 754732, 756457, 757716, 758701, 760423, 764155, 771948, 772180, 773833, 77414, 774790, 776097, 776823, 777635, 7798, 77999, 781054, 784464, 786076, 786116, 786303, 793306, 7942, 79719, 80320, 80493, 807135, 80719, 80936, 81190, 8131, 83688, 849362, 851148, 853123, 853662, 85535, 855765, 85723, 858245, 858881, 861122, 862375, 863117, 863667, 865037, 867261, 868342, 870005, 870545, 8708, 871062, 871407, 872354, 872533, 873772, 873884, 874134, 874254, 877725, 878793, 879435, 87967, 880120, 880647, 881102, 882816, 882877, 883113, 883322, 884026, 888361, 890453, 893519, 89878, 899125, 899442, 9004, 900962, 904781, 908119, 909273, 915022, 916340, 916856, 917472, 918239, 919347, 919444, 922953, 922981, 93499, 936213, 941657, 942555, 95987, 9605, 9748, 99273, 99608
210037, 508083
331781, 636244, 777228
730739
109509
736891
28062, 468803
100255, 100636, 10209, 102324, 102342, 106753, 10725, 108115, 108573, 108900, 109067, 110280, 111256, 111359, 111814, 113856, 113962, 114278, 114811, 115424, 116404, 116689, 117343, 118464, 11909, 120659, 123434, 123918, 126504, 12686, 128856, 12901, 130084, 131124, 132709, 133194, 133295, 133686, 13427, 14502, 145388, 147678, 148234, 148924, 149315, 149466, 149517, 151113, 153130, 153508, 153558, 154207, 155270, 155432, 155518, 155974, 156199, 157316, 158049, 158672, 16022, 16051, 160615, 160827, 160886, 16109, 161117, 162239, 162965, 163411, 164164, 165082, 16613, 166504, 166659, 167290, 167348, 16826, 17301, 17366, 177143, 18028, 180517, 180756, 181029, 181412, 18177, 182433, 182435, 182626, 182754, 183396, 18400, 184113, 190240, 191388, 19558, 195617, 19820, 200634, 201149, 202096, 202317, 20279, 20421, 204576, 206127, 206355, 207534, 208231, 210697, 212018, 21202, 217197, 218123, 220711, 220809, 220903, 221392, 222626, 224629, 225096, 226200, 226762, 227541, 228423, 230007, 230502, 231625, 232028, 232809, 237809, 238036, 239609, 24024, 244196, 24666, 253001, 255217, 256211, 25653, 25680, 257576, 2596, 259723, 262354, 26265, 267707, 269800, 27132, 27628, 278358, 278569, 280249, 284205, 284887, 28700, 290723, 291442, 294025, 295785, 29754, 29890, 299795, 299982, 300386, 302280, 305400, 305815, 306293, 307648, 310803, 311283, 31201, 312342, 313844, 313884, 315951, 317090, 318945, 319821, 323035, 323148, 32382, 325576, 325585, 327374, 328622, 329270, 32967, 329996, 330580, 330773, 331031, 331574, 334485, 334702, 335325, 335683, 336399, 336954, 340055, 34197, 346531, 347750, 348009, 34869, 349423, 350365, 352733, 353519, 354100, 354478, 355411, 356777, 357351, 358993, 359565, 360603, 360806, 362450, 362709, 363596, 365358, 367372, 369072, 371934, 372373, 376406, 377793, 378485, 378549, 379846, 380753, 382138, 382290, 382679, 382994, 386493, 387277, 388020, 391784, 393022, 393726, 393916, 394456, 395860, 397149, 398646, 399888, 40007, 400133, 400563, 400579, 401049, 403229, 403786, 405553, 407016, 408488, 408502, 409972, 411542, 414694, 415615, 415921, 417487, 418185, 418190, 420558, 422164, 42339, 42571, 42617, 426429, 42670, 433979, 435240, 437723, 439894, 440206, 44417, 446442, 447529, 451583, 452797, 454031, 454192, 454265, 454722, 456808, 457856, 458266, 458340, 462583, 464218, 46730, 47043, 475183, 475402, 477592, 478554, 478857, 479109, 480436, 481253, 482244, 483165, 48483, 484899, 48510, 486508, 486869, 487117, 487772, 490401, 490724, 491672, 491866, 492585, 494571, 497423, 497958, 498648, 501211, 501525, 502059, 502859, 505132, 507162, 507565, 509758, 511783, 512263, 513777, 514809, 517635, 521783, 524604, 52539, 526447, 5272, 530252, 532101, 533094, 534035, 535859, 536699, 537314, 539480, 540417, 541255, 544011, 544716, 545574, 546078, 548951, 549672, 553108, 553389, 553730, 55431, 555637, 558427, 559925, 560463, 560656, 560709, 560858, 561297, 564395, 564620, 56544, 568072, 568366, 568756, 569854, 5701, 572245, 574029, 574910, 575850, 576692, 578524, 582226, 58398, 585135, 588999, 592975, 593111, 596476, 597045, 59710, 600011, 60035, 60654, 607155, 60923, 609417, 609520, 612693, 614892, 618118, 62066, 621330, 624781, 626943, 62697, 62710, 630431, 630825, 631323, 631352, 635555, 636523, 636765, 637312, 637544, 637545, 638404, 638788, 643368, 6437, 643723, 644725, 644981, 647749, 648106, 64912, 650784, 652592, 659296, 659942, 660099, 661892, 661976, 662192, 664348, 665502, 667067, 669005, 672282, 672295, 673014, 67482, 674974, 677363, 680711, 680878, 685968, 685975, 686110, 686936, 691424, 691498, 695387, 697590, 699484, 702570, 70307, 703526, 703599, 704531, 706523, 706764, 70730, 708802, 70989, 713543, 714433, 717512, 720715, 723510, 728010, 728307, 728662, 729558, 730303, 730512, 731094, 731118, 732046, 732950, 73440, 735017, 735129, 735489, 736988, 739351, 740691, 741010, 742006, 742175, 748171, 748868, 750642, 753115, 755282, 755634, 756517, 758003, 76114, 76148, 765446, 768343, 769095, 769130, 770564, 773711, 774828, 77589, 776321, 777870, 77791, 77874, 781238, 782158, 782901, 782995, 78324, 783306, 784407, 785731, 786611, 786687, 787745, 79118, 79337, 795856, 79999, 801014, 801399, 80732, 807697, 807961, 81619, 8201, 82431, 82933, 8302, 83059, 83993, 848997, 85075, 859459, 859485, 861808, 864502, 864888, 868115, 868944, 870032, 870190, 872038, 873590, 874795, 874927, 879969, 880849, 881786, 883117, 887251, 887489, 889157, 891242, 893861, 89403, 89980, 899990, 904168, 9061, 90710, 91242, 915183, 918534, 921936, 925081, 928287, 928340, 930712, 931140, 93161, 931783, 932718, 93393, 935613, 936341, 93651, 937565, 93851, 939155, 943434, 9823, 9940, 99415, 9987
211023
10029, 10168, 103796, 106540, 106961, 10813, 109023, 112253, 114751, 117863, 11816, 118384, 11843, 119046, 119273, 122179, 12522, 125410, 125525, 129373, 129443, 131860, 133672, 134084, 134243, 134660, 135102, 13533, 135468, 13947, 148384, 149215, 151073, 151362, 151389, 151822, 152185, 152403, 154058, 154468, 154720, 154853, 154981, 156561, 158563, 159812, 161363, 162668, 16359, 164215, 167094, 17365, 175924, 176956, 177223, 17866, 180758, 181209, 187825, 187866, 18788, 190340, 195013, 195241, 196813, 198940, 20029, 202678, 206121, 211804, 212636, 214986, 21506, 215771, 22343, 224840, 225450, 229161, 230564, 230932, 232059, 232945, 236912, 237134, 237201, 238204, 23958, 240093, 24043, 241419, 24410, 246716, 246973, 254094, 255843, 2572, 257230, 26086, 261097, 261935, 265416, 267361, 26791, 268097, 269089, 269930, 27322, 275083, 275866, 287577, 290290, 292364, 292732, 295038, 295687, 297302, 298970, 301253, 304043, 307652, 31269, 315024, 315042, 31575, 316700, 321698, 321712, 32339, 323551, 324012, 326736, 329173, 329763, 332767, 335450, 337500, 340609, 341763, 346324, 347157, 348493, 34955, 354270, 355274, 356479, 357168, 357294, 357696, 357862, 358253, 358612, 360129, 361923, 362305, 363602, 365032, 365164, 365213, 366304, 36935, 369897, 370039, 371900, 373772, 378804, 379080, 380385, 382260, 382718, 386331, 386801, 389199, 392827, 394938, 395075, 399990, 400644, 403241, 40410, 40747, 40780, 408970, 411276, 411357, 414548, 415695, 421632, 430615, 433635, 434215, 437858, 438634, 443296, 443854, 445532, 446955, 44752, 45453, 454620, 455073, 46068, 461893, 468406, 471593, 479542, 482268, 48490, 485041, 486341, 488868, 492026, 492296, 497708, 502867, 503488, 503503, 503557, 504193, 506247, 509453, 513696, 516612, 520802, 521898, 526641, 527101, 529561, 531915, 533418, 537866, 540090, 542047, 548182, 548200, 552588, 552929, 553129, 553566, 554231, 555871, 556403, 557928, 560466, 561417, 561781, 562374, 56423, 564542, 5669, 569853, 57315, 580372, 582407, 583035, 584018, 588299, 593252, 593516, 606499, 608774, 622735, 625544, 629485, 63220, 632887, 635468, 635643, 635648, 635979, 6395, 643781, 648378, 649400, 650681, 654253, 656777, 659372, 66015, 66491, 665754, 665901, 675582, 682125, 685571, 688540, 68901, 689039, 689232, 691175, 692319, 69636, 698006, 702269, 707846, 708905, 714445, 715850, 718303, 725860, 72734, 728880, 729266, 729482, 730370, 731182, 731598, 734350, 736013, 73648, 738883, 73991, 740473, 741197, 763390, 76464, 766331, 76843, 76996, 771188, 771834, 773141, 773851, 774048, 774973, 77774, 780158, 78151, 79026, 794023, 80849, 81844, 82035, 83958, 849650, 850359, 85124, 854535, 855430, 857351, 859580, 860162, 862708, 864081, 864758, 867654, 868206, 869142, 870567, 870678, 8731, 874123, 874541, 874883, 879761, 88193, 88743, 889337, 889761, 890461, 90014, 912587, 915044, 918420, 918997, 919803, 921435, 925962, 931648, 935172, 935771, 938902, 942528, 96499
122692
100579, 102552, 103203, 103377, 106708, 109246, 10956, 117620, 130144, 131066, 131152, 134043, 134869, 14132, 145772, 15035, 150756, 151681, 154121, 15506, 155803, 156626, 15775, 158029, 158120, 16200, 162378, 163680, 165034, 165914, 165969, 167302, 173564, 17755, 183348, 183681, 184402, 18808, 194422, 194836, 196084, 200487, 203044, 205866, 206528, 211411, 212543, 213736, 214385, 217860, 220424, 223725, 22659, 227468, 227526, 227644, 231321, 231961, 234557, 240088, 241512, 243622, 244402, 245811, 251056, 253784, 261531, 261644, 264487, 264926, 266088, 269734, 272529, 274865, 283301, 288974, 291151, 316128, 316543, 321921, 324281, 324828, 326957, 330451, 331666, 343432, 351507, 35224, 352649, 359424, 359763, 359934, 359940, 36225, 364009, 368075, 36915, 375767, 375950, 377745, 382924, 387380, 39050, 391155, 39628, 39963, 40000, 40025, 404418, 40816, 417574, 426066, 438387, 446585, 44729, 447750, 454006, 454560, 45988, 463618, 465470, 48114, 481768, 48662, 487105, 497709, 503295, 503795, 506912, 512513, 513251, 51839, 519398, 520640, 521499, 5230, 524378, 526062, 542320, 54466, 549240, 554155, 55441, 557615, 560032, 560634, 562164, 567245, 570279, 576176, 584749, 587056, 589802, 603496, 607239, 6263, 628072, 631958, 633143, 634261, 635826, 6371, 639987, 640127, 650593, 650905, 653046, 65806, 65974, 666431, 669640, 672874, 681046, 685744, 687227, 691636, 696002, 699978, 70040, 719917, 720214, 721028, 722355, 723458, 735105, 737795, 749433, 755025, 762513, 766644, 774271, 775216, 779648, 781641, 791354, 80111, 851106, 853758, 855775, 856586, 862030, 870807, 874108, 881865, 882669, 888933, 894422, 899897, 903375, 908382, 912461, 917087, 920942, 922524, 927337, 928272, 93283, 933060, 97021
108051, 108675, 110171, 111217, 111756, 112895, 11405, 117794, 120724, 124325, 130104, 13853, 151387, 156096, 1567, 157562, 157605, 15840, 15918, 164323, 172869, 187546, 192553, 198740, 199683, 200044, 203101, 20815, 209499, 214867, 21810, 225211, 226397, 231380, 234276, 23488, 243122, 243455, 253051, 258339, 258790, 260288, 260413, 263839, 268891, 26965, 272585, 27429, 276312, 27632, 2774, 289738, 296396, 29939, 299683, 304597, 306779, 325493, 331790, 332153, 346292, 347657, 348249, 351938, 35363, 356403, 357590, 361691, 365953, 366629, 369017, 371938, 372376, 374552, 376547, 380018, 385132, 38693, 388112, 388127, 390113, 390839, 399564, 40015, 404563, 406199, 432808, 437593, 440753, 442820, 45361, 462413, 464912, 47288, 487146, 490129, 495429, 4975, 500034, 500510, 50266, 503034, 505880, 509040, 516564, 523782, 527500, 535901, 552399, 557167, 558022, 560570, 563633, 581080, 612290, 616865, 619093, 629461, 63376, 635506, 636753, 636810, 642433, 649083, 683558, 686151, 71293, 720443, 726937, 731264, 736650, 741557, 76126, 768492, 774917, 779211, 789984, 798223, 854811, 859338, 872524, 875274, 888171, 914891, 918395, 919684, 926737, 936691
116812, 124133, 13237, 132885, 148014, 149948, 155776, 17585, 17803, 184801, 187081, 201226, 211340, 214361, 225838, 240151, 242406, 243533, 248137, 25578, 256938, 25734, 259565, 271010, 273870, 28703, 336387, 338172, 340750, 342322, 361055, 368240, 376687, 38767, 392456, 42268, 428053, 440310, 485756, 492001, 500431, 511276, 519299, 566172, 585239, 60874, 688362, 69990, 71853, 727919, 750935, 758011, 759642, 89026, 914266, 931258
106462, 122819, 151376, 152051, 154856, 161479, 165774, 177500, 188930, 191238, 205898, 210452, 210672, 212656, 215927, 229039, 232589, 236973, 239875, 240927, 252283, 256484, 260664, 265753, 266601, 26675, 267488, 276638, 2770, 290786, 296215, 308344, 32111, 322258, 328276, 338727, 34085, 360636, 375702, 394462, 398860, 410786, 411080, 414749, 468014, 47218, 483099, 496519, 500385, 519627, 527650, 571737, 583705, 611457, 615318, 620142, 641719, 645498, 66010, 700053, 711840, 727348, 731876, 870445, 871359, 88949, 893712, 906133, 927241, 940513
622536
10285, 108878, 125786, 12640, 155222, 163671, 176714, 179464, 196298, 213680, 234833, 234947, 267128, 272317, 279571, 298928, 312656, 327504, 328312, 331153, 35092, 387321, 392919, 473787, 478714, 490360, 523816, 566035, 580029, 600295, 635851, 639815, 664843, 672890, 6787, 699836, 725015, 730625, 779971, 859308, 902229
524437
102500, 156572, 162719, 18690, 229399, 232741, 234180, 250280, 265026, 275825, 30539, 312059, 317030, 332853, 384736, 413566, 525282, 553122, 576163, 65432, 658354, 735711, 762572, 773310, 888476, 911019, 97985
518872
110872, 113319, 115524, 149772, 162540, 20757, 216537, 239138, 2545, 269108, 278985, 290588, 298240, 299976, 306651, 313668, 337773, 357494, 379293, 390680, 393868, 474278, 484886, 492385, 506812, 544004, 644978, 650810, 698246, 715492
101004, 114509, 148173, 163629, 166505, 181680, 202252, 217638, 240915, 329101, 373560, 377607, 380430, 443952, 452982, 482136, 488066, 522073, 568802, 731992, 877648, 939992
110031, 181784, 203776, 219443, 252761, 32798, 354326, 35452, 358924, 363731, 372337, 436376, 445643, 472593, 480490, 557664, 578676, 579713, 617980, 6231, 646825, 663592, 67964, 703946, 715001, 738183, 740527, 741865, 867248, 918531, 941738
115539, 157786, 189397, 192338, 216961, 217561, 266770, 26899, 389241, 420562, 42083, 480745, 520590, 583139, 677588, 71169, 716812, 8250, 875962, 87981, 933677, 96511, 99271
89846
179767, 253205, 352981, 410769, 522596
134764, 18089, 353742, 515738, 539715, 648892, 730392, 92880
291257, 374121, 441284, 473900, 599447
268868, 364480, 392215, 505875, 864149
124469, 24004, 295690, 307840, 363758, 657236, 774402
17774, 217134, 377738, 424077
357199, 397044, 438607, 438781, 562449, 566367
102965, 118096, 137950, 163872, 258616, 268528, 356351, 415775, 721521
439081, 731240
162913, 360970
175517
163420, 212550, 255086, 27707, 326592, 356666, 424336, 47833, 489296, 539324, 59676, 63380, 771226, 921248
111215, 16069, 558282
523856, 6896
264430, 388973, 432988, 730956
127343
39018, 565617, 873887
104333
340208, 407110
41685
101219, 12611, 12897, 154525, 159633, 160795, 189769, 195453, 257692, 263843, 267158, 287831, 292535, 328751, 333260, 351516, 363852, 387683, 426893, 44448, 483410, 484594, 48981, 502255, 504356, 50763, 567530, 626062, 627960, 659008, 676319, 682539, 708645, 783613, 862862, 902400, 940243
334149, 466982, 474227, 481466, 507439
132183, 17341, 356133, 371814
128893, 398882
401986, 759920
288411
489352
432055
103751, 103997, 107062, 107504, 108884, 10964, 109704, 11038, 11292, 113941, 114886, 114986, 115595, 115854, 117692, 118644, 118874, 119918, 126027, 12695, 12735, 128881, 130016, 130253, 13074, 133239, 133466, 134265, 134683, 139685, 141464, 147740, 147924, 149221, 151352, 151629, 151798, 152035, 152535, 152905, 153066, 153340, 153342, 153994, 154682, 155441, 155519, 155537, 155815, 155986, 157480, 158047, 158511, 159082, 160549, 160895, 161393, 161498, 162583, 162698, 163819, 163995, 164312, 165915, 166816, 16904, 175479, 176324, 180089, 182811, 183074, 184833, 186230, 186461, 186822, 187327, 187757, 189075, 189878, 190363, 191169, 191383, 191624, 191645, 194160, 195239, 195358, 195517, 195634, 195872, 195936, 196102, 198346, 199158, 199429, 201372, 202319, 203923, 204370, 204445, 205156, 205610, 206394, 206644, 209222, 20964, 209905, 210398, 213679, 213831, 214318, 214321, 214322, 21547, 216140, 217174, 217217, 218226, 218520, 221591, 221948, 222551, 223116, 223569, 224719, 225463, 226181, 226963, 227575, 229366, 229436, 229927, 229958, 231072, 23142, 231456, 232323, 232348, 232386, 233028, 234654, 234799, 234839, 237585, 237658, 23861, 238636, 239147, 23918, 239690, 241043, 241315, 242368, 244344, 244583, 24462, 245012, 246625, 247465, 247598, 247741, 248596, 249579, 251736, 252978, 253559, 256396, 256806, 257906, 258309, 259453, 261360, 262584, 263560, 264568, 266043, 267157, 268239, 26907, 269464, 269799, 270401, 271431, 27162, 271641, 271871, 27378, 273849, 277131, 278093, 278188, 279952, 280846, 281769, 281826, 282575, 282690, 283260, 28349, 284186, 285330, 286240, 286568, 288218, 28867, 289107, 289217, 291217, 293745, 29405, 295860, 296578, 297980, 2989, 299718, 299957, 299964, 300059, 303838, 304019, 306033, 306857, 307177, 307811, 308020, 308657, 309714, 309871, 310400, 310972, 311247, 311749, 312673, 313706, 313990, 314495, 31528, 320779, 321301, 322088, 323215, 326328, 327939, 329046, 329110, 329439, 329488, 329641, 329983, 330452, 331792, 33428, 334880, 335998, 337770, 338720, 338922, 342497, 346307, 346973, 347097, 348695, 351861, 352071, 353204, 354429, 354621, 35598, 358000, 358413, 358743, 359440, 3599, 361533, 36343, 364804, 364943, 366749, 367449, 368022, 36928, 369786, 372627, 376225, 377993, 380693, 38105, 383142, 386843, 38842, 388538, 38898, 392631, 395048, 395588, 396346, 39658, 396751, 400160, 400691, 401211, 401219, 401379, 404035, 40562, 407457, 409576, 412568, 41392, 415523, 4170, 41802, 42119, 427865, 428612, 428785, 435906, 436335, 440454, 441496, 443620, 44447, 445975, 446713, 453242, 453490, 454348, 46181, 464497, 464838, 468147, 472634, 47309, 473672, 478608, 47924, 479317, 480819, 481894, 483053, 484741, 49045, 490646, 494848, 495, 495642, 50097, 503181, 503817, 504332, 504466, 506523, 507409, 510403, 513454, 515878, 520917, 523686, 524433, 525664, 526835, 528722, 529950, 53007, 530138, 530564, 5306, 5319, 533186, 533983, 534946, 536530, 537658, 538109, 538936, 540068, 541636, 54879, 551606, 55233, 555271, 558440, 558671, 559367, 559441, 564235, 567983, 568399, 571356, 571710, 572255, 57892, 5821, 584285, 584819, 585663, 588802, 59314, 594650, 598883, 59919, 600811, 601159, 602025, 60399, 60782, 610270, 612052, 615732, 617732, 618208, 622755, 626505, 630043, 630183, 633397, 634376, 637547, 63766, 638790, 64064, 641226, 642436, 646823, 648075, 64861, 655085, 655888, 659231, 65955, 662662, 663000, 665496, 665619, 66587, 672210, 673899, 675347, 677400, 678269, 68156, 681617, 6817, 68410, 684125, 685640, 688590, 694330, 694859, 697280, 69854, 698589, 701673, 70211, 702875, 704771, 707364, 709326, 710828, 71086, 711276, 715349, 718552, 718882, 720674, 724960, 727936, 73, 731735, 735076, 735215, 738478, 739047, 741174, 746043, 748362, 758056, 759369, 759871, 760333, 76211, 762677, 76485, 766065, 766502, 767964, 768449, 768902, 769918, 771342, 771598, 771693, 775549, 778619, 780636, 787231, 78891, 79019, 79765, 80125, 81021, 81655, 83132, 84102, 84559, 85078, 859439, 860148, 861935, 863813, 867632, 868153, 869128, 872436, 874169, 875250, 875629, 876263, 877142, 877452, 881358, 882439, 88315, 884280, 893973, 896675, 896889, 901190, 90834, 909864, 91212, 919298, 923992, 926770, 930054, 933038, 933059, 933963, 935454, 936092, 937047, 937727, 941081, 941793, 943601, 94783, 95155, 95693, 9699, 97033, 9770, 98992
407936
100521, 101158, 101277, 105592, 106334, 10942, 109602, 110961, 11157, 112180, 112387, 112976, 114686, 115020, 120568, 12813, 129476, 131115, 132670, 135785, 139834, 14055, 149181, 149491, 149589, 150150, 15017, 150424, 150912, 151878, 152292, 152941, 153063, 153780, 154246, 154803, 155048, 155760, 155858, 156291, 156922, 157011, 157257, 159204, 159498, 15980, 160212, 162028, 165591, 166226, 175114, 175304, 175720, 175890, 176478, 182700, 183502, 184167, 184737, 186250, 187376, 188677, 189775, 190612, 191579, 191759, 194156, 195111, 195452, 196878, 197563, 19959, 200318, 201913, 202265, 202628, 202696, 203073, 20393, 204764, 206898, 207160, 20883, 209697, 210423, 214541, 215386, 215949, 216160, 21723, 221087, 221810, 230039, 230772, 232644, 234201, 235839, 23651, 237238, 238428, 238491, 239560, 240625, 243176, 243733, 244814, 244845, 245215, 24810, 250417, 256637, 256813, 259767, 262332, 265463, 265487, 265496, 26627, 266774, 268288, 268989, 271218, 272602, 273609, 274190, 274659, 27565, 27693, 282447, 284754, 290270, 292504, 294678, 297608, 298723, 30067, 30330, 30401, 305342, 307339, 309691, 309901, 310406, 311130, 311223, 31443, 315689, 318772, 319203, 323137, 324488, 324505, 32645, 326909, 328244, 32983, 332326, 335133, 335437, 339376, 340409, 342619, 343804, 345748, 350788, 35479, 355561, 357635, 358169, 360730, 361530, 36336, 364535, 368471, 370932, 373825, 376513, 37667, 377396, 378673, 380570, 380674, 382664, 38895, 390713, 390868, 390966, 392396, 392802, 394294, 394915, 397807, 400324, 404483, 407934, 408597, 410764, 410872, 412085, 412437, 412875, 41334, 414733, 415443, 420592, 421076, 434004, 434495, 435020, 435168, 435210, 440428, 442535, 443505, 44381, 451032, 452251, 462394, 464345, 464739, 464865, 466388, 467106, 467928, 468864, 471582, 474288, 48526, 486529, 488486, 490006, 490361, 490404, 497359, 503821, 513239, 51631, 516602, 51758, 5213, 522425, 52376, 525640, 526129, 531142, 5364, 536849, 536954, 537745, 538741, 545635, 546222, 548552, 550748, 55278, 561252, 561802, 563137, 565682, 56888, 572787, 573316, 580349, 583459, 585142, 590077, 590351, 593662, 59523, 598239, 602340, 614200, 616429, 624466, 626056, 629710, 6327, 63414, 635168, 639296, 64036, 643460, 643744, 645477, 645915, 65148, 655602, 663116, 663572, 664023, 666695, 670734, 67269, 677602, 678830, 679700, 690410, 691417, 692768, 694261, 695956, 696123, 701365, 702639, 703095, 703105, 706011, 706487, 706827, 706955, 70813, 71252, 713520, 71362, 716651, 718075, 71834, 729595, 7317, 737175, 73764, 739416, 742017, 743425, 745844, 752609, 755553, 758218, 758741, 759470, 763607, 771197, 777465, 782112, 786281, 78923, 79522, 79807, 80094, 81821, 8244, 83311, 84026, 85045, 85217, 852785, 853569, 858101, 860985, 861098, 861911, 862252, 863705, 864109, 86411, 86558, 869568, 873895, 874730, 8754, 880721, 881484, 882114, 889389, 891718, 891853, 895273, 904398, 905797, 907698, 90906, 910323, 910800, 915678, 919258, 919284, 92450, 926508, 926678, 927764, 931426, 932233, 933913, 935366, 935729, 936968, 93740, 937882, 938156, 938803, 939626, 94123, 943294, 97821, 98758, 99604
118449
101515, 102868, 105395, 107827, 108257, 109585, 11027, 110413, 113655, 11512, 116539, 116837, 117050, 117427, 118722, 127198, 130155, 140083, 149483, 151963, 152802, 153554, 154323, 156607, 157250, 158100, 159105, 161355, 162850, 163011, 165315, 166997, 175051, 180660, 181746, 18200, 183454, 187150, 198229, 198535, 205602, 210021, 211746, 215360, 216175, 216256, 217092, 218568, 220736, 229032, 231839, 23285, 233170, 234797, 236974, 238268, 238668, 239003, 240805, 244681, 249759, 263509, 268648, 269058, 274258, 274389, 275826, 282227, 283361, 288179, 290189, 292370, 296831, 301674, 302304, 304005, 310755, 31343, 315624, 320305, 328233, 328890, 329068, 335873, 336665, 337139, 337775, 339405, 339824, 342769, 344993, 345741, 346641, 355571, 355644, 357791, 358622, 361958, 367199, 370395, 375624, 376575, 381059, 386275, 386916, 388643, 388684, 390012, 393914, 394540, 397317, 397836, 397866, 405016, 405905, 408829, 40921, 41162, 42635, 430210, 436609, 438550, 439754, 440233, 447151, 453562, 455205, 46782, 470317, 484586, 485053, 488219, 488667, 496718, 497437, 498507, 499947, 502732, 503850, 506488, 507681, 516226, 516945, 523812, 528113, 533339, 540047, 547903, 551605, 55422, 556593, 557884, 562527, 563393, 567341, 567685, 573401, 585201, 5864, 60337, 6059, 615646, 619942, 629517, 630939, 636111, 638477, 638751, 648441, 658197, 66050, 662811, 669918, 675225, 678596, 679909, 679978, 68251, 687733, 705117, 709335, 713401, 714069, 716924, 720318, 723361, 726031, 729142, 729510, 735996, 73716, 74068, 745971, 748159, 748374, 748858, 749499, 75151, 753550, 754225, 757642, 767487, 769525, 772805, 773588, 77850, 785854, 786818, 8000, 850828, 853403, 853857, 85418, 863020, 868826, 870954, 871688, 878479, 879734, 883042, 88381, 914287, 917436, 919164, 922830, 928432, 928861, 930289, 938546, 94870, 96240, 97206, 9915
107111, 128470, 382712
923210
102417, 10968, 110134, 112830, 113374, 11426, 114320, 117016, 118042, 118552, 121725, 125504, 130059, 130625, 133027, 133136, 134924, 149467, 150196, 150892, 151007, 151430, 152598, 152599, 154071, 154549, 155586, 155688, 16114, 161152, 161720, 167403, 176899, 179508, 179929, 181195, 183600, 184552, 185901, 187970, 188400, 196674, 199256, 199672, 204420, 205160, 207952, 209456, 209468, 210247, 218256, 223169, 225301, 226983, 230970, 232454, 234054, 23531, 238087, 238417, 238606, 23915, 243813, 245022, 247971, 248680, 251793, 256265, 257009, 257902, 258589, 25904, 259192, 260061, 262077, 268568, 271234, 273959, 274924, 280693, 290807, 29294, 293668, 294079, 294277, 295172, 298265, 299804, 302362, 30413, 308201, 311720, 316086, 316533, 316760, 326073, 328667, 332179, 334493, 343852, 344183, 345582, 348199, 348588, 351362, 35998, 360562, 364597, 364726, 369996, 370741, 371952, 374023, 375625, 377021, 378166, 382069, 383020, 384006, 385716, 388750, 38960, 389937, 390458, 391768, 391812, 392807, 392821, 394384, 394751, 397921, 398753, 403666, 408746, 418100, 426128, 435731, 43815, 439791, 439854, 443747, 445966, 447391, 448595, 464882, 472129, 476219, 480877, 481810, 486256, 486265, 49231, 493840, 494697, 498649, 501290, 502473, 503873, 516992, 517088, 525382, 527301, 534045, 535426, 536951, 543665, 544843, 549742, 559360, 560620, 560884, 562845, 574074, 579745, 583550, 590248, 590497, 590838, 591215, 592351, 599502, 605299, 605529, 608414, 620455, 620496, 622062, 635200, 635575, 6388, 644972, 645385, 65653, 657184, 660895, 66226, 662308, 671698, 673951, 674025, 677761, 681883, 68210, 689759, 689816, 701211, 701413, 702006, 705468, 714025, 715694, 72525, 72545, 725470, 72844, 730632, 738481, 745677, 74704, 750002, 75463, 762929, 76303, 767154, 769302, 771968, 774674, 775043, 783317, 82369, 856596, 857334, 860170, 861368, 871384, 873788, 893470, 901448, 902767, 903806, 908100, 9091, 909684, 91742, 921628, 92180, 922363, 922915, 92409, 930679, 930774, 933053, 93632, 936744, 9550, 96399, 9766
100177, 10038, 101948, 109674, 112987, 113990, 115932, 116529, 119029, 119652, 123878, 130087, 133101, 133200, 14393, 150219, 150854, 151804, 152633, 153046, 153670, 155892, 16553, 16686, 166915, 175109, 179721, 186677, 188345, 196034, 198822, 202580, 202915, 204213, 205950, 208268, 218909, 226308, 227019, 228783, 230524, 235922, 237811, 240435, 241645, 242493, 24571, 247642, 250196, 250642, 253585, 255817, 257938, 264360, 26469, 269450, 271487, 272623, 274838, 278496, 282536, 292558, 296852, 297742, 306176, 306461, 308951, 311772, 312334, 317481, 318476, 321213, 323511, 325892, 326667, 327856, 328186, 329767, 333218, 337895, 345071, 357823, 358941, 359795, 360181, 360521, 362741, 365995, 366075, 367271, 368552, 369670, 372978, 373250, 383716, 389471, 391410, 39145, 392136, 397155, 397248, 401955, 412351, 412623, 415167, 42962, 435432, 440681, 441314, 449416, 456734, 469349, 471315, 471504, 472435, 472957, 47921, 486623, 488927, 488987, 496565, 504255, 504950, 511389, 516120, 531894, 531980, 537589, 544918, 555623, 557781, 560434, 560705, 561076, 562089, 563928, 574172, 577324, 60650, 612422, 620223, 62437, 639814, 639932, 640290, 648461, 651937, 666956, 667918, 670368, 674343, 674484, 69, 703390, 712482, 713711, 716184, 717032, 73359, 736663, 737373, 75136, 757299, 757855, 757908, 759238, 759491, 763850, 766007, 770844, 774520, 785208, 790542, 854065, 864130, 869056, 873805, 878105, 878839, 898068, 9064, 912379, 922052, 923257, 926561, 933819, 935721, 937798, 937898, 939400, 939504, 940475, 943337, 98506, 99843
205588
10379, 104245, 104710, 105504, 107067, 109796, 110232, 111042, 114286, 116660, 120528, 124851, 125626, 126549, 132722, 133518, 150633, 151405, 152157, 153179, 153330, 156585, 158209, 158799, 159846, 160435, 160951, 163424, 16541, 166072, 176588, 182134, 185684, 186433, 187117, 188967, 191180, 191269, 195061, 199010, 202818, 203389, 205430, 205645, 212937, 214866, 215751, 218067, 222082, 223279, 232799, 238403, 238553, 247209, 247310, 249372, 253173, 262718, 263083, 263150, 266773, 28248, 290268, 297702, 302264, 307804, 310917, 311354, 314063, 319992, 320486, 323593, 331963, 341809, 344488, 351617, 353876, 35460, 355875, 360859, 361059, 361880, 365045, 3688, 380362, 383390, 390131, 391369, 399861, 405863, 40696, 4120, 41693, 41707, 438024, 44072, 441791, 442524, 446582, 453127, 453351, 454952, 473019, 475577, 478156, 478340, 503276, 507616, 5204, 536023, 562993, 563824, 565606, 56681, 569122, 569188, 58085, 583371, 592586, 59965, 60110, 618210, 647454, 648490, 654112, 655653, 655821, 660160, 664571, 666334, 666869, 671335, 67656, 678552, 686842, 69736, 699246, 712507, 71610, 716816, 729652, 731652, 736371, 742340, 752116, 752624, 756002, 769564, 775920, 776569, 77989, 784722, 80020, 81641, 85158, 861404, 861908, 863084, 86582, 867233, 867770, 868802, 872486, 885491, 886552, 888206, 88917, 892864, 89291, 899342, 912541, 915358, 921252, 923919, 924641, 925765, 926050, 936419, 937961, 938513, 9774, 9852, 99605
585494
32534, 9611
467469, 663214
661741
709764
117624, 124300, 127314, 128017, 132604, 139876, 14237, 144270, 145777, 150677, 151693, 15496, 157258, 167111, 17552, 186556, 195487, 195893, 197781, 20844, 21277, 21626, 229837, 236828, 239953, 244075, 246375, 251535, 258804, 269805, 291260, 298926, 30059, 315411, 317898, 321678, 326155, 32632, 326910, 326974, 356726, 369289, 408694, 410027, 41536, 45328, 454343, 454884, 45700, 466045, 481097, 509755, 519518, 520831, 5210, 541707, 542901, 549815, 553830, 557128, 569348, 56985, 574087, 581896, 589105, 598625, 599934, 603499, 60445, 61075, 611743, 621940, 622003, 66151, 673367, 675644, 677943, 68870, 694817, 702815, 713432, 732201, 732990, 738659, 74314, 746872, 748756, 751637, 753739, 758965, 760210, 763461, 770597, 78035, 81670, 868759, 871545, 879788, 88419, 897586, 923767, 92838, 93530
182366
101317, 103455, 107730, 148931, 179440, 182873, 185528, 191404, 194482, 198807, 201176, 203052, 207068, 22217, 232515, 232765, 233087, 236729, 253638, 267216, 26727, 277436, 28447, 2874, 288114, 308450, 311674, 327274, 346085, 355316, 367895, 378321, 415977, 432364, 44055, 442972, 454183, 481586, 484944, 495243, 497098, 497731, 50974, 541860, 563566, 565489, 581048, 591407, 602954, 631584, 637388, 650275, 67141, 691056, 71622, 718207, 724782, 764338, 7703, 781067, 852000, 861320, 98680
294459
105369
103925, 104445, 111499, 126536, 132703, 134261, 141232, 15093, 154025, 154097, 160406, 176923, 194462, 207139, 212706, 21387, 22077, 225145, 230823, 232899, 233347, 233490, 23417, 241791, 245131, 246908, 25158, 277045, 294525, 306824, 333878, 336544, 347874, 356013, 356790, 35751, 369117, 371249, 376419, 38116, 392799, 401094, 405562, 416593, 42486, 468737, 487437, 491570, 501603, 512379, 52809, 562665, 58776, 596129, 654565, 663212, 675510, 727791, 736894, 740506, 744994, 748947, 754465, 759186, 762295, 764056, 8124, 8385, 8481, 84818, 8559, 876948, 914125, 940351
154046, 667193
622179, 916594
617909, 84262
61246
100985, 101788, 107444, 115465, 117611, 133711, 148514, 155288, 165744, 17808, 183741, 187479, 195739, 20887, 21004, 218673, 226827, 23482, 238891, 245581, 245945, 247264, 259918, 270549, 291471, 300131, 311264, 313831, 330679, 334014, 34045, 35145, 362954, 365051, 368832, 380115, 436381, 471641, 495602, 513763, 51644, 561322, 574854, 586083, 591299, 5960, 59916, 601922, 611719, 634242, 646712, 66262, 666063, 695923, 7155, 735656, 760430, 77603, 77613, 78065, 79638, 80586, 84910, 857557, 905964, 91947, 919799, 922131, 940596, 942108, 94567
10471, 11776, 132677, 148396, 154132, 157510, 160275, 167183, 204205, 20637, 212207, 214497, 230955, 23226, 237673, 240886, 249535, 26941, 269834, 288434, 293771, 295816, 29596, 297866, 300738, 314423, 331653, 334846, 356439, 362766, 372466, 373015, 387427, 3898, 39568, 401706, 403128, 40894, 41786, 428375, 472359, 514620, 519990, 522427, 531488, 534567, 549057, 555086, 562575, 567272, 605684, 608845, 610231, 6148, 633298, 643561, 657402, 660438, 668808, 675886, 688074, 690893, 691076, 701705, 703936, 70849, 721639, 726895, 745036, 765958, 777531, 788302, 808384, 85613, 878106, 885881, 886396, 917470, 919307, 919596, 927405, 932977, 941261
106327, 114729, 151299, 152132, 152560, 15479, 157082, 162229, 163949, 19026, 193487, 19537, 197741, 198379, 208014, 20812, 216882, 231379, 235201, 235762, 243610, 249291, 285082, 298495, 323678, 404185, 421795, 430363, 47007, 482392, 50504, 506276, 529790, 542095, 562874, 685686, 725031, 872616, 93773
161069
283715
602975, 674245, 933134
485188
12469, 187048, 210277, 231338, 232628, 23553, 2449, 245696, 24653, 255838, 273943, 311072, 335503, 3420, 344868, 357940, 35850, 381496, 41798, 431262, 442143, 491177, 537653, 568450, 571175, 660904, 66280, 677195, 704949, 729583, 74023, 74667, 749898, 881830, 88701, 906504, 918525, 926653, 930471
12154, 144032, 153177, 188994, 199814, 23309, 277380, 281906, 293053, 307964, 315207, 328983, 359691, 411660, 484605, 50470, 554362, 570278, 616951, 629986, 650094, 709787, 712146, 740767, 879990, 882010, 91630, 92602
284820, 516357
574124, 889483
454257
111036, 114125, 117910, 127129, 149588, 156812, 157536, 15958, 160195, 203040, 216924, 218141, 243480, 265707, 266077, 290181, 293579, 3086, 311599, 320429, 34819, 369204, 369613, 39078, 484228, 509491, 513254, 523602, 532814, 540165, 582954, 59330, 604663, 659459, 683277, 692186, 692262, 70350, 719860, 726938, 782070, 82669, 874744, 88029, 919246, 921396, 929914, 932711, 933092
49766
102338, 114715, 13213, 132443, 15198, 160048, 183850, 187025, 192589, 207898, 22712, 227141, 238341, 249896, 27247, 322265, 335007, 349420, 386063, 392974, 424824, 449002, 45131, 491126, 524, 533249, 635949, 642072, 64565, 701593, 712154, 733652, 81311, 861781, 862883, 876227, 909815, 9259, 9797
252769, 26775, 30335, 321735, 360152, 365886, 497366, 562288, 596356, 721922, 722200, 79015
153394, 16461, 165972, 17243, 201803, 212728, 213187, 23326, 244564, 246925, 255101, 287705, 301727, 368751, 398464, 407173, 420713, 499294, 509654, 59305, 595167, 604910, 611648, 63311, 640710, 669712, 694135, 70760, 707829, 735302, 769443, 771761, 88857, 89689, 9714
104404, 116455, 152908, 161784, 18428, 196142, 199570, 212682, 245866, 253676, 277344, 373281, 398391, 450484, 653847, 656439, 65972, 716682, 742488, 766424, 858067, 914662, 924802
149339, 166906, 175635, 18093, 182718, 184578, 200665, 201444, 223484, 264957, 273113, 312150, 348219, 39953, 446072, 552168, 61156, 687731, 696541, 712862, 74233, 9163, 926248, 96575
11012, 123013, 25415, 276069, 340687, 36283, 43039, 483759, 514514, 599128, 641029, 661759, 99611
134422, 149942, 157682, 158439, 166923, 181074, 181895, 204027, 22740, 230096, 29491, 309346, 331660, 352335, 38036, 39121, 654209, 663483, 696302, 75059, 773755, 875768, 899901, 98523
113499, 133366, 182023, 213333, 254466, 259339, 26644, 269414, 271440, 344332, 357845, 614163, 658910, 70911, 718085, 897543, 930279, 930355, 96156
13311, 153041, 189816, 199705, 30528, 306886, 330999, 356430, 403606, 462609, 468287, 546059, 587615, 63272, 669953, 720644, 728486, 927398, 942167
115996, 211979, 234546, 238694, 239672, 294845, 332674, 513392, 69119, 727324, 78119, 848929, 850920, 8842
128519, 14342, 162142, 180745, 181806, 22920, 251265, 304225, 311555, 318246, 364269, 370879, 380193, 526368, 593804, 628449
163800, 195220, 19753, 20326, 21221, 22970, 254818, 307724, 387902, 401475, 48812, 517873, 534953, 60768, 629477, 65890, 701990, 707119, 70992, 732911
11469, 175551, 201377, 251626, 251935, 27640, 284618, 35304, 37953, 38110, 449288, 476268, 769620, 776163, 89875
118305, 123769, 213854, 214581, 240765, 254337, 374135, 489128, 52668, 69800, 753038
111183, 13616, 17400, 177096, 18249, 233541, 250925, 27609, 361866, 388414, 46709, 534355, 546502, 560196, 57574, 583352, 643101, 718681, 88474, 931288, 932542, 938334
187128, 229218, 35569, 526761, 597534, 861752
156327, 156898, 478874, 585675, 664528, 728993
100320, 225139, 40945, 44611, 505494, 657109, 680, 723097, 82749, 904061
149827, 152245, 162392, 192720, 207116, 213615, 236792, 237746, 244732, 24755, 253383, 299914, 364905, 404833, 410548, 42032, 47078, 51619, 524386, 550513, 562388, 64144, 643958, 704012, 752445, 919036, 99789
150670, 183766, 20996, 233736, 65752, 678112, 728953, 75795, 914300, 929604
152539, 324825, 41811, 691587, 695701, 69908, 891120
187429, 200225, 231346, 396336, 488213, 583763, 720961, 748888, 906868, 929904
135206, 219952, 42795, 855762
247936, 275838, 316942, 5190, 530386, 57552
112466, 437464, 714105, 852528, 896300
160179, 264798
229871, 688582, 782468
24404, 30103, 55161, 570619
2055, 257316, 39015, 5722, 596307
15041, 728530
191179, 36682, 651638
110964, 211706, 294554, 3603, 368974, 467887, 471062, 544390, 626368, 67723, 723450, 728036, 76186, 926826, 96140
23098
103310, 110264, 15104, 15205, 362490, 79469
14334, 347791, 516200, 679502
13701, 233859, 588803
10749, 739058
734630, 81848
654085
16703, 215139, 375318, 92350
14814, 150139, 160546, 163706, 184597, 19927, 21456, 262732, 29484, 328869, 336018, 357131, 400881, 438665, 463586, 468162, 489454, 493527, 522471, 634799, 645165, 723721, 861272
344209, 519378, 718580
14826, 209124, 273837
13906
160845
192660, 332848, 439859
60618, 912656
118776, 744221
101853, 104089, 105624, 105629, 105968, 106557, 106893, 108621, 110238, 11095, 112052, 112133, 112784, 114785, 114989, 11548, 116335, 117337, 122954, 127642, 131081, 132843, 13304, 133179, 134384, 135546, 13818, 148915, 150063, 150177, 150915, 151115, 152998, 153114, 157824, 15911, 1608, 16130, 161425, 162340, 163227, 166137, 166796, 175163, 176604, 180095, 182213, 182884, 182889, 183556, 183765, 184397, 184555, 185071, 185552, 18651, 187337, 187801, 188484, 188799, 188968, 18922, 190866, 193652, 195769, 195798, 196649, 19819, 200637, 201709, 202290, 204310, 207455, 208992, 209291, 21397, 214138, 214368, 21446, 215284, 217749, 21827, 219456, 223469, 224920, 226706, 229083, 230941, 232956, 237429, 238999, 23951, 239526, 239726, 24687, 248026, 248027, 248377, 25330, 253650, 253741, 254381, 254619, 255187, 261003, 263705, 264425, 26450, 26458, 264641, 267771, 268066, 268178, 273781, 273961, 274271, 276124, 279727, 280433, 281616, 283319, 291363, 296807, 297715, 300623, 306800, 307182, 323607, 323702, 325712, 325884, 330988, 332265, 332622, 334221, 334518, 334557, 336361, 337294, 337900, 341720, 344512, 347895, 349972, 355770, 356639, 356659, 357701, 362021, 362880, 365198, 36742, 367663, 374074, 378328, 380040, 385514, 385665, 386173, 386829, 389860, 392504, 392937, 39295, 393235, 393984, 3972, 397505, 400171, 40127, 406142, 4092, 40968, 413942, 417340, 428423, 432502, 437114, 438990, 440138, 441426, 451619, 453048, 456073, 457215, 467104, 467942, 468824, 47842, 479528, 481071, 483594, 487234, 492301, 492728, 500677, 502273, 504795, 505965, 507421, 508474, 508519, 51050, 512729, 52032, 522580, 527890, 530376, 5333, 534739, 539972, 546852, 5472, 54826, 549296, 552841, 55458, 56200, 563251, 566191, 571277, 573751, 576742, 577736, 584256, 58814, 590801, 593743, 596970, 599898, 600637, 603944, 639180, 641205, 645009, 646234, 648215, 650804, 65882, 658900, 663122, 668417, 670148, 674062, 677652, 679031, 681683, 686234, 688843, 690122, 690137, 692542, 693717, 704445, 70468, 705450, 705523, 70783, 708513, 713521, 714863, 715235, 716982, 720396, 722087, 729167, 731097, 732676, 734744, 737918, 739829, 74751, 748280, 751982, 75377, 756506, 757003, 75969, 76109, 76183, 767952, 770288, 77254, 7734, 773774, 777157, 7777, 781899, 782368, 78293, 80095, 81698, 83535, 849644, 85292, 856477, 858503, 859159, 861041, 86212, 864257, 864805, 866737, 866892, 867464, 869198, 869826, 870829, 87164, 872366, 872735, 875230, 875475, 875846, 87610, 877357, 877369, 877802, 879525, 880355, 884299, 8845, 888091, 892410, 897952, 899380, 906749, 912332, 914404, 917835, 921102, 921620, 92389, 924352, 925313, 931324, 93166, 937132, 938969, 95034, 95862, 96135, 96459, 9918, 99580, 9978
192882
534316
151774, 209887, 292408, 448405, 5024, 709558
510387, 624629
15267
101640, 103158, 105183, 106068, 106575, 10720, 107262, 107610, 108113, 111291, 111310, 111805, 112433, 113076, 113496, 113964, 114297, 114649, 114957, 114959, 114967, 11533, 117054, 118240, 118310, 118890, 118912, 118994, 12214, 123610, 123840, 126673, 12697, 127207, 129491, 12967, 130295, 13259, 132823, 133121, 134122, 134338, 134873, 135192, 135354, 136657, 137411, 140064, 14531, 14564, 147803, 148539, 148639, 149746, 149774, 151307, 151883, 152621, 152639, 152912, 153847, 154660, 155, 155455, 156045, 156488, 156538, 157238, 157340, 157784, 160107, 16066, 161056, 161702, 162428, 162468, 162499, 162701, 162998, 163267, 16347, 163536, 164085, 164184, 165040, 165672, 173163, 17354, 175187, 175428, 176151, 176707, 176726, 179315, 179836, 181325, 184515, 185257, 186075, 186592, 187442, 187849, 188488, 190141, 190179, 190447, 191077, 19113, 193061, 193172, 193335, 194037, 194881, 195127, 196769, 199055, 200636, 201349, 201603, 202648, 202873, 20313, 203853, 204338, 205020, 205540, 205822, 206404, 20714, 208308, 20923, 209696, 209704, 209710, 209749, 210914, 211092, 212489, 214083, 215251, 21554, 215821, 217511, 218129, 220931, 223822, 224800, 226708, 227027, 227188, 227507, 22764, 228698, 22911, 229137, 22935, 229423, 229621, 229663, 230104, 230172, 230280, 230625, 230688, 231324, 231912, 233628, 234135, 234156, 234508, 234677, 234689, 234743, 234963, 235135, 238631, 239662, 239824, 241068, 241989, 242925, 243100, 243466, 244320, 24439, 244527, 246081, 246087, 246569, 246576, 246819, 247117, 247202, 247980, 24799, 248203, 248732, 249199, 249450, 249991, 251915, 252586, 253033, 253091, 253558, 253619, 254261, 254827, 255875, 256890, 257711, 258768, 258894, 259278, 26004, 260267, 261316, 261461, 262135, 263677, 264512, 264780, 265523, 265561, 266777, 267447, 268059, 268163, 269097, 269603, 269839, 27089, 27153, 272342, 272483, 27348, 275815, 27862, 279845, 282354, 283331, 283503, 284485, 286093, 287020, 287094, 287535, 288464, 288519, 288664, 290084, 293868, 294564, 294757, 295215, 295680, 295865, 297078, 297988, 298740, 299554, 299658, 299813, 299967, 300657, 300672, 300905, 305054, 306287, 306457, 307376, 307553, 308766, 309001, 31003, 31073, 311370, 311703, 313515, 313653, 314384, 314493, 31544, 31563, 317348, 318831, 319335, 320456, 320464, 320797, 321191, 325057, 32758, 328642, 328774, 329302, 330046, 330098, 330645, 330990, 331763, 331987, 332468, 33287, 33327, 333955, 334404, 335220, 335281, 337006, 337577, 340859, 341677, 342360, 343942, 346813, 346966, 347293, 34857, 348720, 348846, 349778, 349892, 350055, 350134, 350514, 350570, 350966, 351165, 351181, 351875, 352388, 353003, 355893, 358021, 358134, 358860, 359643, 359679, 361131, 361173, 361452, 361634, 362513, 363649, 364072, 365352, 365642, 365677, 370005, 370804, 372259, 375018, 375101, 375107, 377194, 378550, 378980, 380202, 38081, 381282, 381385, 382430, 383665, 384075, 384448, 386221, 388238, 388346, 388940, 388981, 389151, 389231, 389555, 390166, 390507, 391158, 391457, 392366, 392400, 392524, 392712, 393013, 393155, 395428, 395463, 396168, 396307, 397061, 398678, 399842, 400530, 400559, 401704, 403034, 403998, 404647, 405281, 407009, 407044, 40836, 409926, 414055, 417527, 417716, 418184, 419552, 419649, 420303, 420452, 422017, 425510, 426037, 42722, 428491, 42870, 429018, 429835, 43121, 436594, 437532, 440122, 440364, 442938, 444476, 444895, 445379, 445804, 446604, 453928, 456821, 46103, 462062, 462474, 463747, 467075, 46726, 471170, 473119, 474400, 474650, 47545, 477825, 47832, 478352, 478811, 479119, 479555, 479693, 483838, 485591, 48651, 488525, 490279, 493073, 493203, 494112, 495472, 498696, 499751, 500504, 501259, 501541, 50185, 502910, 502953, 503136, 50360, 503602, 504626, 507331, 508575, 509766, 510963, 514822, 515834, 516046, 519645, 522691, 523168, 528244, 528605, 52914, 531572, 532283, 533236, 534686, 539295, 541097, 54141, 543146, 546418, 552086, 552190, 553166, 554045, 557168, 557321, 557684, 55921, 560267, 560338, 560609, 561049, 565918, 566663, 566694, 567278, 568334, 570011, 570519, 571381, 579068, 585199, 585746, 585960, 589230, 592307, 596439, 597592, 599498, 600209, 600279, 600421, 601307, 602089, 60233, 602949, 603065, 603157, 604350, 606337, 606886, 608381, 609250, 609914, 61039, 611147, 611578, 613389, 614816, 614899, 615777, 617264, 617583, 617902, 620342, 621068, 623442, 624124, 625578, 627718, 628962, 629355, 629478, 629559, 634282, 634487, 634536, 634663, 638980, 639140, 641239, 642358, 643109, 643657, 64414, 645816, 648019, 649229, 649359, 651294, 652300, 653347, 654306, 655610, 657589, 660327, 661497, 662067, 662163, 666111, 667395, 667813, 668258, 668761, 669058, 671094, 671118, 672103, 672894, 673590, 674117, 674307, 677331, 678024, 679721, 679922, 680472, 682818, 686214, 687114, 687360, 687389, 687853, 688436, 688676, 690234, 690513, 690618, 691588, 695525, 699099, 700912, 701172, 707449, 709833, 71032, 71104, 712361, 712813, 713049, 71546, 716332, 716394, 716810, 717578, 718845, 718959, 719059, 71936, 720018, 721189, 722716, 72562, 725809, 727810, 732411, 73290, 733215, 7347, 740654, 744390, 744394, 744431, 744974, 746354, 747790, 748103, 75392, 756290, 756777, 757878, 761162, 76139, 762577, 764506, 766856, 767600, 768698, 768986, 769646, 770615, 771619, 772974, 77635, 777425, 780406, 78163, 782653, 78362, 784737, 785016, 785592, 802654, 81927, 82771, 84218, 847234, 84921, 851895, 853214, 854141, 856741, 857390, 859203, 861825, 861957, 861968, 864290, 864796, 865952, 868846, 870053, 870150, 87246, 875756, 880723, 882016, 882694, 884064, 885748, 886728, 888813, 888935, 88992, 88995, 88997, 89200, 892877, 89321, 895317, 897104, 900654, 90071, 903953, 906925, 907863, 908850, 913785, 914607, 914844, 917277, 917347, 920574, 922421, 925434, 925571, 927245, 927881, 927932, 928692, 929589, 930143, 932747, 933557, 935145, 935580, 937528, 938852, 941249, 942180, 94239, 943486, 94504, 95244, 95483, 96313, 97697, 98824, 99861
385154, 469143
391883
18363, 224934
101760, 10376, 103972, 105068, 109682, 110150, 128311, 128872, 155857, 163541, 180064, 185764, 189266, 195570, 202325, 20466, 21231, 21439, 233366, 233689, 233703, 234174, 234254, 239573, 241580, 248700, 252430, 25377, 259029, 268585, 269238, 270015, 270311, 278379, 299768, 309783, 312784, 313952, 315051, 316144, 320687, 333588, 33573, 340003, 34922, 353166, 357656, 361254, 362047, 378630, 381640, 392989, 398857, 407673, 411009, 434990, 444466, 456200, 483101, 497910, 503115, 503837, 521299, 525334, 525861, 548446, 55354, 563640, 568428, 570052, 575652, 580595, 61074, 611949, 615713, 621156, 641809, 641941, 648366, 650004, 659468, 702114, 710030, 715039, 718064, 733169, 742544, 746943, 751002, 766886, 783172, 85286, 87100, 875206, 87926, 881495, 888717, 897794, 90303, 96190, 98801
395564, 450810, 71682
202015
111289, 15389
422089, 746205
49386
227000
367961
719791
188981, 244991, 277354, 328551, 336436, 453229, 523636, 562812, 564185, 577628, 594011, 666632, 70124, 770351, 77900, 786683, 877333
451006, 453155, 466586, 499436, 599943, 624206, 651798, 672376, 69440, 7287, 737964, 767887, 888978, 892163, 917754
492221, 538191, 613960, 853558, 95254
434871, 49149, 539734, 656488
509935
274422, 659258
249602, 297935, 448412, 504503, 535738, 591069, 84616, 919030
395084, 655855
570060
103778, 245813, 282643
246187, 470116, 483554, 503902, 696676, 885570
271526
681571
28355, 488273
528304
223476, 61453
259871
290442
680706
114072, 17825, 178578, 244747, 305281, 362437, 366646, 3843, 455891, 65899, 66177, 9112, 915096
587204
622034
855203, 889193

"""




# Split by commas and newlines, clean up whitespace
numbers = [n.strip() for n in data.replace("\n", ",").split(",") if n.strip()]

# Convert to integers
numbers = [int(n) for n in numbers]
# Count them
count = len(numbers)

print(f"Total count: {count}")

Total count: 8271


In [14]:
data_df = ret_metabase("egypt", 64814)
data_df = data_df[data_df["COUPON_VALUE"].notnull()].copy()
coupons = data_df[["RETAILER_ID", "COUPON_VALUE"]].rename(columns={
    "RETAILER_ID": "retailer_id",
    "COUPON_VALUE": "amount"
})
google_sheets("On_Ground_Dispatch_LOGS", "Coupons", "append", df=coupons)

/home/ec2-user/service_account_key.json
/home/ec2-user/service_account_key.json


'Data is appended to the sheet successfully'

In [15]:
df = google_sheets("On_Ground_Dispatch_LOGS", "Coupons", "get")
df

/home/ec2-user/service_account_key.json


,retailer_id,amount
0,80048.0,50.0
1,50142.0,50.0
2,84968.0,50.0
3,732810.0,50.0
4,428632.0,50.0
...,...,...
145,39408.0,50.0
146,46475.0,50.0
147,58956.0,50.0
148,880158.0,50.0


In [16]:
google_sheets("On_Ground_Dispatch_LOGS", "Coupons", "clear")

/home/ec2-user/service_account_key.json


'sheet is cleared'

In [8]:
tasks_df = dwh_query("egypt", "select * from MAIN_EGYPT_LOGISTICS.Tasks where status_id = 5 order by 8 desc", "read")
tasks_df

/home/ec2-user/service_account_key.json


,id,customer_id,type_id,status_id,visit_id,date,description,created_at,updated_at,workflow_id,workflow_type_id,version,retailer_id
0,6889109,166056,NaN,5,5887564.0,2025-10-13,None,2025-10-13 16:21:24.314028,2025-10-13 16:21:47.574572,1.0,1,2,166056
1,6889106,416163,NaN,5,5887561.0,2025-10-13,None,2025-10-13 16:19:35.726929,2025-10-13 16:21:03.046208,1.0,1,2,416163
2,6889102,616179,NaN,5,5887557.0,2025-10-13,None,2025-10-13 16:18:00.205517,2025-10-13 16:18:37.084081,1.0,1,2,616179
3,6889100,920751,NaN,5,5887555.0,2025-10-13,None,2025-10-13 16:16:55.950407,2025-10-13 16:19:18.076937,1.0,1,2,920751
4,6889099,575843,NaN,5,5887554.0,2025-10-13,None,2025-10-13 16:16:55.332106,2025-10-13 16:17:11.051518,1.0,1,2,575843
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637399,146481,485030,2.0,5,139963.0,2024-01-02,,2024-01-02 09:32:18.000000,2024-01-02 13:17:41.000000,NaN,1,0,485030
1637400,146475,275145,2.0,5,139957.0,2024-01-02,,2024-01-02 09:32:18.000000,2024-01-02 12:46:06.000000,NaN,1,0,275145
1637401,146500,209742,2.0,5,139982.0,2024-01-02,,2024-01-02 09:32:18.000000,2024-01-02 12:16:58.000000,NaN,1,0,209742
1637402,146316,11169,2.0,5,139998.0,2024-01-02,,2024-01-02 09:32:18.000000,2024-01-02 13:06:38.000000,NaN,1,0,11169


In [33]:
    today_df = dwh_query("Egypt", """
select TO_CHAR(created_at, 'Dy') as day_of_week, 
   TO_CHAR(DATE_TRUNC('hour', created_at) + ((CEIL(EXTRACT(MINUTE FROM created_at) / 30.0) * 30) || ' minutes')::interval, 'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where cast(so.created_at as date) = current_date
group by 1,2 
order by 2 desc
""", "read", columns=[], conn=None)
today_df.head(25)

/home/ec2-user/service_account_key.json


,day_of_week,half_hour_label,order_count
0,Mon,15:30,137
1,Mon,15:00,158
2,Mon,14:30,160
3,Mon,14:00,122
4,Mon,13:30,119
5,Mon,13:00,112
6,Mon,12:30,91
7,Mon,12:00,75
8,Mon,11:30,79
9,Mon,11:00,66
